In [1]:
import sys
sys.executable

'D:\\ProgramData\\Anaconda3\\python.exe'

In [2]:
import shapefile
import shapely.geometry as geometry
import pandas as pd
import geopandas
from matplotlib import pyplot as plt
import pyproj
from pandas.core.frame import DataFrame
import haversine as hs
from geopy.distance import geodesic
import math

In [3]:
# calculate distance, time, average speed, accerleration
def cal(rt):
    # calculate distance: default unit-km
    geoToList = list(rt['geometry_cal'])
    dist=[0]
    for i in range(len(geoToList)-1):
        loc1=eval(geoToList[i])
        loc2=eval(geoToList[i+1])
        dist1=hs.haversine(loc1,loc2)
        dist.append(dist1)
        #dist unit: km
    df_dist = pd.DataFrame(dist, columns =['dist']).reset_index(drop=True)
    rt=rt.reset_index(drop=True)
    result=pd.concat([rt,df_dist],axis=1)
    
    #calculate time
    timeList = list(rt['tmstmp'])
    interval=[0]
    for i in range(len(timeList)-1):
        t1=pd.to_datetime(timeList[i])
        t2=pd.to_datetime(timeList[i+1])
        ts=t2-t1
        interval.append(ts.total_seconds())
        #duration unit: s
    df_ts = pd.DataFrame(interval, columns =['duration']).reset_index(drop=True)
    
    #calculate speed 
    #spd unit: mph
    speedList = list(rt['spd'])
    spd_change=[0]
    for i in range(len(speedList)-1):
        spd1=speedList[i]
        spd2=speedList[i+1]
        spd_diff=spd2-spd1
        spd_change.append(spd_diff)
    df_spd = pd.DataFrame(spd_change, columns =['speed_change']).reset_index(drop=True)
    df_ts = pd.DataFrame(interval, columns =['duration']).reset_index(drop=True)

    result2=pd.concat([result,df_ts],axis=1)
    result1=pd.concat([result2,df_spd],axis=1)
    
    #calculate average speed and accerlerate speed
    # average speed unit:km/s to m/s
    result1["aver_v"]=1000*result1['dist']/result1["duration"]
    #accelerate v 
    #mph to m/s first
    #加速度单位： m/s-2
    result1['a']=result1["speed_change"]*0.44704/result1['duration']
    
    #calculate sin & cos values of the slope
    SlopeToList = list(rt['road_angle'])
    sin_value=[]
    cos_value=[]
    for i in range(len(SlopeToList)-1):
        sin1=math.sin(math.radians(SlopeToList[i]))
        cos1=math.cos(math.radians(SlopeToList[i]))
        sin_value.append(sin1)
        cos_value.append(cos1)
    df_sin = pd.DataFrame(sin_value, columns =['sin']).reset_index(drop=True)
    df_cos = pd.DataFrame(cos_value, columns =['cos']).reset_index(drop=True)
    result2=pd.concat([result1,df_cos],axis=1)
    result1=pd.concat([result2,df_sin],axis=1)
    
    #Froll=Mgf.cosa: g=9.81 f=0.01
    #Fclimb=Mg.sina
    #assume average passenger mass = 70kg, bus mass=12000kg
    result1['total_mass']=12000+result1['pass_mass']*70
    #1N=1kg.ms-2
    result1["Froll"]=result1['total_mass']*9.81*0.01*result1['cos']
    result1["Fclimb"]=result1['total_mass']*9.81*result1['sin']
    result1['Fdrag']=0.5*1.2*0.6*7.5*result1['aver_v']*result1['aver_v']
    result1['Finertia']=1.1*result1['total_mass']*result1['a']
    result1['Ftr']=result1["Froll"]+result1["Fclimb"]+result1['Fdrag']+result1['Finertia']
    
    result1 = pd.concat([result1,pd.DataFrame(columns=['n'])], sort=False)

    #needs to be calculated differently depending on whether the tractive force is positive (energy demand) or negative (energy regeneration
    result1["n"][result1.Ftr>=0]=1/(0.97*0.95*0.91)
    result1["n"][result1.Ftr<0]=0.97*0.95*0.91*0.6

    #calculate the energy consumption during the trip Etr based on the a sign
    result1 = pd.concat([result1,pd.DataFrame(columns=['Etr'])], sort=False)

    #根据加速度分类讨论
    #能量单位从焦耳到千瓦时:除以3.6*10^6
    #a+:
    #单位：J
    result1["Etr_a+"]=result1["Froll"]+result1["Fclimb"]+0.5*1.2*0.6*7.5*result1["a"]*result1["dist"]+result1["Finertia"]
    #Kd:0.5*1.2*0.6*7.5
    #the dist unit should transfer to m!!
    result1["Etr"][result1.a>0]=result1["n"]*result1["dist"]*1000*result1["Etr_a+"]

    #a=0
    result1["Etr_a0"]=result1["Froll"]+result1["Fclimb"]+0.5*1.2*0.6*7.5*result1['aver_v']*result1['aver_v']
    result1["Etr"][result1.a==0]=result1["n"]*result1["dist"]*result1["Etr_a0"]*1000

    #a<0
    result1["Etr_a-"]=result1["Froll"]+result1["Fclimb"]-0.5*1.2*0.6*7.5*result1["a"]*result1["dist"]+result1["Finertia"]
    result1["Etr"][result1.a<0]=result1["n"]*result1["dist"]*result1["Etr_a-"]*1000
    
    #单位：kwh
    result1["Etr"]=result1["Etr"]/3600000
    
    return result1

In [4]:
def OD_identify(intersect):
    spdList=list(intersect["spd"])
    dest=[]
    
    for i in range(len(spdList)):
        if spdList[i-1]!=0 and spdList[i]==0 and spdList[i+1:i+10]==[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]:
            dest.append('D')
        elif spdList[i]!=0 and spdList[i-9:i]==[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] and spdList[i+1]!=0:
            dest.append('O')
        else: 
            dest.append('0')
    
    #append one OD identify column
    dest = pd.DataFrame(dest, columns =['O/D']).reset_index(drop=True)
    intersect=pd.concat([intersect,dest],axis=1)
    
    return intersect

# first step to split by journey 
def OD_identify2(intersect1,df):
    OD1=OD_identify(intersect1)
    OD=OD1[['id','O/D']]#replace OD2=pd.concat([OD2_1,OD2_2]) because of round trip
    #append the OD info to the origin datasets
    rt_OD = pd.merge(df, OD, how='left', on=['id'])
    rt_OD.set_index(["id"], inplace=True)
    rt_O=rt_OD[rt_OD['O/D']=="O"]
#identify the first origin
    rt_start=rt_O.iloc[[0],[42]].values[0][0]
#identify the last dest
    rt_D=rt_OD[rt_OD['O/D']=="D"]
    rt_end=rt_D.iloc[[-1],[42]].values[0][0]
    
    return rt_OD.loc[rt_start:rt_end]

# 定义通用的获取某元素在列表中第n次出现的位置下标的函数
def get_index(l, x, n):
    if n <= l.count(x):
        all_index = [key for key, value in enumerate(l) if value == x]
        return all_index[n]#change[n-1] to n
    else:
        return None

change of OD identify algorithm

In [5]:
def OD_identify(intersect):
    spdList=list(intersect["spd"])
    dest=[]
    
    for i in range(len(spdList)):
        if spdList[i-1]!=0 and spdList[i]==0 and spdList[i+1:i+5]==[0.0, 0.0, 0.0, 0.0]:
            dest.append('D')
        elif spdList[i]!=0 and spdList[i-4:i]==[0.0, 0.0, 0.0, 0.0]:
            dest.append('O')
        else: 
            dest.append('0')
    
    #append one OD identify column
    dest = pd.DataFrame(dest, columns =['O/D']).reset_index(drop=True)
    intersect=pd.concat([intersect,dest],axis=1)
    
    return intersect

# first step to split by journey 
def OD_identify2(intersect1,df):
    OD1=OD_identify(intersect1)
    OD=OD1[['id','O/D']]#replace OD2=pd.concat([OD2_1,OD2_2]) because of round trip
    #append the OD info to the origin datasets
    rt_OD = pd.merge(df, OD, how='left', on=['id'])
    rt_OD.set_index(["id"], inplace=True)
    rt_O=rt_OD[rt_OD['O/D']=="O"]
#identify the first origin
    rt_start=rt_O.iloc[[0],[42]].values[0][0]
#identify the last dest
    rt_D=rt_OD[rt_OD['O/D']=="D"]
    rt_end=rt_D.iloc[[-1],[42]].values[0][0]
    
    return rt_OD.loc[rt_start:rt_end]

# 定义通用的获取某元素在列表中第n次出现的位置下标的函数
def get_index(l, x, n):
    if n <= l.count(x):
        all_index = [key for key, value in enumerate(l) if value == x]
        return all_index[n]#change[n-1] to n
    else:
        return None

# rt2

In [6]:
df1=pd.read_csv("rt2.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

<ipython-input-3-b4c410bad7e4>:82: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  result1 = pd.concat([result1,pd.DataFrame(columns=['Etr'])], sort=False)
<ipython-input-3-b4c410bad7e4>:82: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  result1 = pd.concat([result1,pd.DataFrame(columns=['Etr'])], sort=False)
<ipython-input-3-b4c410bad7e4>:82: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  result1 = pd.concat([result1,pd.DataFrame(columns=['Etr'])], sort=False)
<ipython-input-3-b4c410bad7e4>:82: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions wit

2101.0    40245
1807.0     1091
1607.0      253
1103.0       11
Name: vid, dtype: int64

In [7]:
df2=df3[df3['vid']==2101]
df2=df2[3000:30000]

In [8]:
df2

Unnamed: 0     vid             tmstmp        lat        lon  \
vid                                                                         
2101.0 3000     255850.0  2101.0  20221103 08:24:10  29.666499 -82.310609   
       3001     255907.0  2101.0  20221103 08:24:21  29.666529 -82.310431   
       3002     255964.0  2101.0  20221103 08:24:37  29.666521 -82.309155   
       3003     256021.0  2101.0  20221103 08:24:56  29.666502 -82.306222   
       3004     256078.0  2101.0  20221103 08:25:07  29.666492 -82.304375   
...                  ...     ...                ...        ...        ...   
       29995   2827536.0  2101.0  20221130 15:57:22  29.646122 -82.322787   
       29996   2827587.0  2101.0  20221130 15:57:37  29.646122 -82.322787   
       29997   2827638.0  2101.0  20221130 15:57:54  29.646123 -82.322785   
       29998   2827689.0  2101.0  20221130 15:58:10  29.646123 -82.322785   
       29999   2827791.0  2101.0  20221130 15:58:27  29.646123 -82.322787   

                hdg    pid   rt            des    pdist  ...     Froll  \
vid                                                      ...             
2101.0 3000   357.0  352.0  2.0  Walmart/Duval  43862.0  ...  1232.136   
       3001    90.0  352.0  2.0  Walmart/Duval  43904.0  ...  1204.668   
       3002    90.0  352.0  2.0  Walmart/Duval  44311.0  ...  1232.136   
       3003    90.0  352.0  2.0  Walmart/Duval  45241.0  ...  1239.003   
       3004    90.0  352.0  2.0  Walmart/Duval  45827.0  ...  1197.801   
...             ...    ...  ...            ...      ...  ...       ...   
       29995  356.0  352.0  2.0  Walmart/Duval    229.0  ...  1197.801   
       29996  356.0  352.0  2.0  Walmart/Duval    229.0  ...  1204.668   
       29997  356.0  352.0  2.0  Walmart/Duval    229.0  ...  1177.200   
       29998  356.0  352.0  2.0  Walmart/Duval    229.0  ...  1218.402   
       29999  356.0  352.0  2.0  Walmart/Duval    229.0  ...  1232.136   

              Fclimb       Fdrag     Finertia          Ftr         n  \
vid                                                                    
2101.0 3000      0.0    7.529639 -3269.808339 -2030.142700  0.503139   
       3001      0.0    6.787336  5489.651200  6701.106536  1.192513   
       3002      0.0  160.435023  5790.285600  7182.856623  1.192513   
       3003      0.0  600.595264  2615.042829  4454.641094  1.192513   
       3004      0.0  710.717131  1637.507520  3546.025651  1.192513   
...              ...         ...          ...          ...       ...   
       29995     0.0    0.000000     0.000000  1197.801000  1.192513   
       29996     0.0    0.000000     0.000000  1204.668000  1.192513   
       29997     0.0    0.000563     0.000000  1177.200563  1.192513   
       29998     0.0    0.000000     0.000000  1218.402000  1.192513   
       29999     0.0    0.000242     0.000000  1232.136242  1.192513   

                   Etr       Etr_a+       Etr_a0       Etr_a-  
vid                                                            
2101.0 3000  -0.008085 -2037.690480  1239.665639 -2037.654198  
       3001   0.038675  6694.338337  1211.455336  6694.300063  
       3002   0.286909  7022.561163  1392.571023  7022.282037  
       3003    0.36179  3854.189845  1839.598264  3853.901814  
       3004   0.167621  2835.367269  1908.518131  2835.249771  
...                ...          ...          ...          ...  
       29995       0.0  1197.801000  1197.801000  1197.801000  
       29996       0.0  1204.668000  1204.668000  1204.668000  
       29997  0.000096  1177.200000  1177.200563  1177.200000  
       29998       0.0  1218.402000  1218.402000  1218.402000  
       29999  0.000066  1232.136000  1232.136242  1232.136000  

[27000 rows x 42 columns]

In [535]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1234"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df2=DataFrame(final_output2)#将字典转换成为数据框
final_df2

,energy(kwh),dist(km),start,end,duration(min)
0,23.068582,22.566810,2022-11-03 08:30:02,2022-11-03 09:28:30,58.466667
1,25.981397,22.760369,2022-11-03 09:30:17,2022-11-03 10:23:02,52.750000
2,27.381619,22.521665,2022-11-03 10:30:03,2022-11-03 11:25:14,55.183333
3,28.990694,23.115617,2022-11-03 11:30:03,2022-11-03 12:27:26,57.383333
4,23.428619,21.507375,2022-11-03 12:30:04,2022-11-03 14:25:33,115.483333
...,...,...,...,...,...
71,21.450626,22.237297,2022-11-29 09:30:02,2022-11-29 10:26:32,56.500000
72,19.470232,22.535673,2022-11-29 10:30:02,2022-11-29 14:25:30,235.466667
73,-0.085997,0.285268,2022-11-30 07:25:10,2022-11-30 07:25:16,0.100000
74,45.115841,41.081732,2022-11-30 07:30:09,2022-11-30 09:25:16,115.116667


In [536]:
final_df4=pd.concat([final_df3,final_df2])
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,28.275345,22.866164,2022-12-09 06:30:02,2022-12-09 07:27:20,57.300000
1,26.173734,22.855617,2022-12-09 08:30:13,2022-12-09 09:24:58,54.750000
2,26.416705,22.955579,2022-12-09 09:30:10,2022-12-09 10:24:12,54.033333
0,23.068582,22.566810,2022-11-03 08:30:02,2022-11-03 09:28:30,58.466667
1,25.981397,22.760369,2022-11-03 09:30:17,2022-11-03 10:23:02,52.750000
...,...,...,...,...,...
71,21.450626,22.237297,2022-11-29 09:30:02,2022-11-29 10:26:32,56.500000
72,19.470232,22.535673,2022-11-29 10:30:02,2022-11-29 14:25:30,235.466667
73,-0.085997,0.285268,2022-11-30 07:25:10,2022-11-30 07:25:16,0.100000
74,45.115841,41.081732,2022-11-30 07:30:09,2022-11-30 09:25:16,115.116667


In [537]:
#this function varies depending on the intervals of bus!!
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 80)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df2=clean_df(final_df2)
final_df2['rt']='2'
final_df2['stop_id']='1234'
final_df2

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/1031523939.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['rt']='2'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/1031523939.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['stop_id']='1234'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,23.068582,22.566810,2022-11-03 08:30:02,2022-11-03 09:28:30,58.466667,2,1234
1,25.981397,22.760369,2022-11-03 09:30:17,2022-11-03 10:23:02,52.750000,2,1234
2,27.381619,22.521665,2022-11-03 10:30:03,2022-11-03 11:25:14,55.183333,2,1234
3,28.990694,23.115617,2022-11-03 11:30:03,2022-11-03 12:27:26,57.383333,2,1234
5,19.048628,20.740468,2022-11-04 06:30:09,2022-11-04 07:25:30,55.350000,2,1234
6,26.427157,23.019936,2022-11-04 07:30:10,2022-11-04 08:27:35,57.416667,2,1234
7,22.216831,20.734419,2022-11-04 08:30:31,2022-11-04 09:25:00,54.483333,2,1234
9,21.527270,22.767293,2022-11-04 11:33:45,2022-11-04 12:29:00,55.250000,2,1234
11,27.827289,23.226736,2022-11-07 06:30:14,2022-11-07 07:25:30,55.266667,2,1234
13,23.577528,22.937535,2022-11-07 09:30:01,2022-11-07 10:25:12,55.183333,2,1234


In [538]:
final_df2.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt2_new.csv",index=False,sep=',')

# rt 1 

1. 读取csv group by vid

In [6]:
df2=pd.read_csv("rt1.csv")
df3=df2.groupby(by='vid').apply(cal)
#vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1203    6600
804     6052
708     6044
1013    1822
2111    1671
1807    1112
1104    1043
705      702
2575     288
707      263
Name: vid, dtype: int64

In [9]:
df2=df3[df3['vid']==1203]
df2=df2[0:6000]

In [10]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

,energy(kwh),dist(km),start,end,duration(min)
0,23.992890,17.446984,2022-12-05 06:01:25,2022-12-05 06:52:04,50.650000
1,22.589906,17.584184,2022-12-05 07:00:21,2022-12-05 08:00:39,60.300000
2,23.746650,17.578319,2022-12-05 08:10:09,2022-12-05 09:08:32,58.383333
3,22.963347,17.595882,2022-12-05 09:25:21,2022-12-05 10:25:30,60.150000
4,23.792348,17.009598,2022-12-05 10:40:07,2022-12-05 14:29:12,229.083333
5,24.571921,17.645595,2022-12-05 14:41:19,2022-12-05 15:44:13,62.900000
6,25.097838,17.648549,2022-12-05 16:02:10,2022-12-05 17:12:46,70.600000
7,24.402454,17.691489,2022-12-05 17:23:12,2022-12-05 18:24:58,61.766667
8,53.822814,35.397900,2022-12-06 14:42:23,2022-12-06 17:12:22,149.983333
9,26.901795,17.564402,2022-12-08 07:01:44,2022-12-08 07:58:25,56.683333


In [600]:
final_df4=pd.concat([final_df3,final_df4])
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,22.230357,17.573482,2022-12-05 06:29:24,2022-12-05 07:23:34,54.166667
1,24.174250,17.621889,2022-12-05 07:29:23,2022-12-05 08:42:06,72.716667
2,23.084453,17.572989,2022-12-05 08:45:47,2022-12-05 09:54:28,68.683333
3,48.175950,34.748450,2022-12-05 09:59:14,2022-12-05 15:08:53,309.650000
4,24.821737,17.700753,2022-12-05 15:19:12,2022-12-05 16:37:46,78.566667
5,24.583201,17.691603,2022-12-05 16:40:28,2022-12-05 17:55:42,75.233333
6,26.715975,17.774129,2022-12-06 14:01:48,2022-12-06 15:09:57,68.150000
7,27.203673,17.620470,2022-12-06 15:19:52,2022-12-06 16:38:42,78.833333
8,24.861696,17.832206,2022-12-06 16:41:31,2022-12-06 18:04:16,82.750000
9,27.721382,17.661197,2022-12-08 06:28:56,2022-12-08 07:22:56,54.000000


In [601]:
#this function varies depending on the intervals of bus!!
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 80)& (rt["duration(min)"]>40)&(rt["energy(kwh)"] < 30)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='1'
final_df4['stop_id']='1493'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/664841416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='1'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/664841416.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='1493'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,22.230357,17.573482,2022-12-05 06:29:24,2022-12-05 07:23:34,54.166667,1,1493
1,24.174250,17.621889,2022-12-05 07:29:23,2022-12-05 08:42:06,72.716667,1,1493
2,23.084453,17.572989,2022-12-05 08:45:47,2022-12-05 09:54:28,68.683333,1,1493
4,24.821737,17.700753,2022-12-05 15:19:12,2022-12-05 16:37:46,78.566667,1,1493
5,24.583201,17.691603,2022-12-05 16:40:28,2022-12-05 17:55:42,75.233333,1,1493
6,26.715975,17.774129,2022-12-06 14:01:48,2022-12-06 15:09:57,68.150000,1,1493
7,27.203673,17.620470,2022-12-06 15:19:52,2022-12-06 16:38:42,78.833333,1,1493
9,27.721382,17.661197,2022-12-08 06:28:56,2022-12-08 07:22:56,54.000000,1,1493
10,27.629388,17.584487,2022-12-08 07:32:04,2022-12-08 08:33:21,61.283333,1,1493
11,25.441566,17.621441,2022-12-08 08:46:03,2022-12-08 09:44:57,58.900000,1,1493


In [602]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt1_new.csv",index=False,sep=',')

# rt 6

In [606]:
df1=pd.read_csv("rt6.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1805.0    19944
1806.0    13503
1807.0     4452
1017.0     1386
2113.0      691
2108.0      588
1803.0      479
2004.0      418
1103.0      156
1012.0       61
2201.0       16
712.0        10
Name: vid, dtype: int64

In [689]:
df2=df1[df1['vid']==1805]
df2=df2[11000:12000]

In [690]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1455"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

KeyError: 'O'

In [684]:
final_df4=pd.concat([final_df3,final_df4])
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,24.195394,22.889382,2022-11-01 14:28:20,2022-11-01 15:27:46,59.433333
1,65.516426,70.678243,2022-11-01 15:29:36,2022-11-01 18:25:44,176.133333
2,43.895212,47.121555,2022-11-02 07:29:31,2022-11-02 09:27:34,118.050000
3,46.737982,47.023220,2022-11-02 09:40:28,2022-11-02 11:25:27,104.983333
4,11.484569,12.580384,2022-11-02 11:30:09,2022-11-02 15:24:49,234.666667
5,25.512914,22.640551,2022-11-03 06:29:45,2022-11-03 07:20:43,50.966667
6,24.911326,23.589549,2022-11-03 07:26:45,2022-11-03 08:27:10,60.416667
7,23.752776,23.404472,2022-11-03 08:28:41,2022-11-03 09:23:11,54.500000
8,48.818667,47.404127,2022-11-03 09:29:54,2022-11-03 11:23:32,113.633333
9,26.117932,23.537432,2022-11-03 11:28:53,2022-11-03 12:24:06,55.216667


In [691]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 80)& (rt["duration(min)"]>40)]
    return rt_filtered

final_df2=clean_df(final_df4)
final_df2['rt']='6'
final_df2['stop_id']='1455'
final_df2

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/2894685503.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['rt']='6'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/2894685503.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['stop_id']='1455'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,24.195394,22.889382,2022-11-01 14:28:20,2022-11-01 15:27:46,59.433333,6,1455
5,25.512914,22.640551,2022-11-03 06:29:45,2022-11-03 07:20:43,50.966667,6,1455
6,24.911326,23.589549,2022-11-03 07:26:45,2022-11-03 08:27:10,60.416667,6,1455
7,23.752776,23.404472,2022-11-03 08:28:41,2022-11-03 09:23:11,54.500000,6,1455
9,26.117932,23.537432,2022-11-03 11:28:53,2022-11-03 12:24:06,55.216667,6,1455
12,26.270306,23.333237,2022-11-03 17:29:56,2022-11-03 18:24:56,55.000000,6,1455
13,23.331702,28.507620,2022-11-03 18:29:08,2022-11-03 19:24:56,55.800000,6,1455
14,24.406153,23.491015,2022-11-21 06:29:36,2022-11-21 07:22:56,53.333333,6,1455
15,20.519511,23.741117,2022-11-21 07:34:03,2022-11-21 08:21:44,47.683333,6,1455
19,26.867161,23.393787,2022-11-29 06:29:35,2022-11-29 07:25:53,56.300000,6,1455


In [692]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt6_new.csv",index=False,sep=',')

# rt 3

In [604]:
df1=pd.read_csv("rt3.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
#vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2103.0    25472
Name: vid, dtype: int64

In [ ]:
df2=df3[df3['vid']==1203]
df2=df2[0:6000]

# rt5

In [6]:
df2=pd.read_csv("rt5.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
vidlist

[1803,
 701,
 1206,
 1106,
 1811,
 1203,
 1101,
 1810,
 1104,
 1802,
 1016,
 1807,
 705,
 1013,
 1012,
 702,
 803,
 1105,
 1806]

In [72]:
df2=df3[df3['vid']==701]


#df2=pd.concat([df4,df5])
df2


Unnamed: 0    vid             tmstmp        lat        lon    hdg  \
vid                                                                           
701 0            3.0  701.0  20221101 14:09:33  29.657475 -82.394900  113.0   
    1           61.0  701.0  20221101 14:09:48  29.656549 -82.392353  112.0   
    2          121.0  701.0  20221101 14:10:04  29.656243 -82.391507  112.0   
    3          180.0  701.0  20221101 14:10:20  29.656158 -82.391273  112.0   
    4          240.0  701.0  20221101 14:10:25  29.656021 -82.390892  112.0   
...              ...    ...                ...        ...        ...    ...   
    1108     58883.0  701.0  20221101 19:08:00  29.652019 -82.329556   90.0   
    1109     58914.0  701.0  20221101 19:08:14  29.652016 -82.329253   90.0   
    1110     58945.0  701.0  20221101 19:08:36  29.651990 -82.326894   91.0   
    1111     58976.0  701.0  20221101 19:08:44  29.651977 -82.326172   91.0   
    1112     59007.0  701.0  20221101 19:08:59  29.651974 -82.326015   90.0   

            pid   rt                  des    pdist  ...        Froll  \
vid                                                 ...                
701 0     335.0  5.0  Rosa Parks/Downtown   5160.0  ...  1239.003000   
    1     335.0  5.0  Rosa Parks/Downtown   6039.0  ...  1222.671504   
    2     335.0  5.0  Rosa Parks/Downtown   6330.0  ...  1231.612258   
    3     335.0  5.0  Rosa Parks/Downtown   6411.0  ...  1224.748177   
    4     335.0  5.0  Rosa Parks/Downtown   6543.0  ...  1211.122959   
...         ...  ...                  ...      ...  ...          ...   
    1108  335.0  5.0  Rosa Parks/Downtown  26692.0  ...  1211.535000   
    1109  335.0  5.0  Rosa Parks/Downtown  26790.0  ...  1232.136000   
    1110  335.0  5.0  Rosa Parks/Downtown  27541.0  ...  1177.200000   
    1111  335.0  5.0  Rosa Parks/Downtown  27770.0  ...  1184.067000   
    1112  335.0  5.0  Rosa Parks/Downtown  27820.0  ...          NaN   

               Fclimb       Fdrag      Finertia           Ftr         n  \
vid                                                                       
701 0        0.000000         NaN           NaN           NaN       NaN   
    1     7974.029737  853.630084  -4094.588373   5955.742952  1.192513   
    2     3592.171284   82.676775 -11194.552160  -6288.091844  0.503139   
    3     3572.151221    6.337003   4222.544260   9025.780660  1.192513   
    4     3159.484805  171.685824  13360.684480  17902.978068  1.192513   
...               ...         ...           ...           ...       ...   
    1108     0.000000    0.000000      0.000000   1211.535000  1.192513   
    1109     0.000000   11.804861   7058.633874   8302.574735  1.192513   
    1110     0.000000  289.870952   1341.120000   2808.190952  1.192513   
    1111     0.000000  205.830074  -4451.512560  -3061.615486  0.503139   
    1112          NaN    2.755679  -6141.882560           NaN       NaN   

               Etr        Etr_a+        Etr_a0        Etr_a-  
vid                                                           
701 0          NaN           NaN           NaN           NaN  
    1     0.450789   5101.898251  10050.331325   5102.327484  
    2     -0.07883  -6370.962314   4906.460316  -6370.574924  
    3     0.073235   9019.463998   4803.236400   9019.423317  
    4     0.234185  17731.398118   4542.293588  17731.186371  
...            ...           ...           ...           ...  
    1108       0.0   1211.535000   1211.535000   1211.535000  
    1109  0.080396   8290.810255   1243.940861   8290.729493  
    1110  0.190163   2518.382532   1467.070952   2518.257468  
    1111 -0.031897  -3267.508792   1389.897074  -3267.382328  
    1112       NaN           NaN           NaN           NaN  

[1113 rows x 42 columns]

In [73]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df2=DataFrame(final_output2)#将字典转换成为数据框
final_df2

C:\Users\zheng\AppData\Local\Temp/ipykernel_23300/1839956441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id'] = range(len(df2))
C:\Users\zheng\AppData\Local\Temp/ipykernel_23300/1839956441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id1'] = range(len(df2))


IndexError: list index out of range

In [71]:
final_df4=pd.concat([final_df4,final_df2])
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,23.617452,19.577269,2022-11-30 06:19:20,2022-11-30 07:12:19,52.983333
1,27.374536,19.448550,2022-11-30 07:20:04,2022-11-30 08:08:45,48.683333
2,23.500890,19.344647,2022-11-30 08:20:05,2022-11-30 09:12:00,51.916667
3,23.882250,19.178628,2022-11-30 09:24:05,2022-11-30 10:22:21,58.266667
4,0.000000,0.000000,2022-11-30 10:36:52,2022-11-30 10:36:52,0.000000
...,...,...,...,...,...
0,25.937588,18.858061,2022-11-01 14:11:47,2022-11-01 15:10:15,58.466667
1,25.447615,19.292265,2022-11-01 15:29:21,2022-11-01 16:31:26,62.083333
2,50.912241,38.629448,2022-11-01 16:38:08,2022-11-01 18:50:53,132.750000
3,28.310964,18.335870,2022-11-01 19:08:49,2022-11-01 20:00:07,51.300000


In [43]:
final_df2

,energy(kwh),dist(km),start,end,duration(min)
0,29.148335,19.219847,2022-11-01 14:28:31,2022-11-01 15:38:33,70.033333
1,23.655543,19.240289,2022-11-01 15:40:47,2022-11-01 16:37:51,57.066667
2,0.020615,0.011472,2022-11-01 16:43:04,2022-11-01 16:43:04,0.000000
3,25.831778,19.353073,2022-11-01 16:50:18,2022-11-01 17:55:41,65.383333
4,24.553987,19.314688,2022-11-07 12:29:59,2022-11-07 13:34:36,64.616667
5,26.553686,19.290762,2022-11-07 13:42:44,2022-11-07 14:42:02,59.300000
6,42.149923,38.207329,2022-11-07 14:52:32,2022-11-07 06:41:35,-490.950000
7,23.943461,19.211486,2022-11-07 06:50:12,2022-11-07 07:39:01,48.816667
8,0.050775,0.032009,2022-11-07 07:46:32,2022-11-07 07:46:32,0.000000
9,23.428243,19.695563,2022-11-07 07:49:47,2022-11-07 08:42:39,52.866667


In [74]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 75)& (rt["duration(min)"]>40)&(rt["energy(kwh)"] < 30)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='1'
final_df4['stop_id']='175'
final_df4=final_df4.drop_duplicates()
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23300/3755014596.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='1'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23300/3755014596.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='175'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,23.617452,19.577269,2022-11-30 06:19:20,2022-11-30 07:12:19,52.983333,1,175
1,27.374536,19.448550,2022-11-30 07:20:04,2022-11-30 08:08:45,48.683333,1,175
2,23.500890,19.344647,2022-11-30 08:20:05,2022-11-30 09:12:00,51.916667,1,175
3,23.882250,19.178628,2022-11-30 09:24:05,2022-11-30 10:22:21,58.266667,1,175
6,22.468008,18.745051,2022-11-30 14:12:05,2022-11-30 15:06:28,54.383333,1,175
...,...,...,...,...,...,...,...
2,23.591088,19.211653,2022-11-02 07:43:28,2022-11-02 08:33:24,49.933333,1,175
4,25.366566,18.831145,2022-11-02 09:50:36,2022-11-02 10:51:07,60.516667,1,175
0,25.937588,18.858061,2022-11-01 14:11:47,2022-11-01 15:10:15,58.466667,1,175
1,25.447615,19.292265,2022-11-01 15:29:21,2022-11-01 16:31:26,62.083333,1,175


In [75]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt5_new1.csv",index=False,sep=',')

# rt 7

In [693]:
df1=pd.read_csv("rt7.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2105.0    41372
1103.0      310
711.0       118
Name: vid, dtype: int64

In [763]:
df2=df3[df3['vid']==2105]
df2=df2[25000:34000]

In [764]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="326"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

,energy(kwh),dist(km),start,end,duration(min)
0,25.642470,19.434197,2022-11-21 07:30:39,2022-11-21 08:22:25,51.766667
1,26.253259,19.245808,2022-11-21 08:31:51,2022-11-21 09:20:31,48.666667
2,28.286558,19.309145,2022-11-21 09:30:06,2022-11-21 10:21:01,50.916667
3,29.738817,19.373728,2022-11-21 10:31:04,2022-11-21 14:23:31,232.450000
4,27.200955,19.348746,2022-11-21 14:30:26,2022-11-21 15:28:11,57.750000
5,27.120204,18.876091,2022-11-21 15:31:53,2022-11-21 16:22:55,51.033333
6,25.086894,19.391452,2022-11-21 16:33:06,2022-11-21 17:27:02,53.933333
7,24.785911,18.782910,2022-11-21 17:30:24,2022-11-21 18:20:37,50.216667
8,25.916431,18.902449,2022-11-22 06:29:46,2022-11-22 07:22:31,52.750000
9,27.908071,19.338353,2022-11-22 07:30:40,2022-11-22 08:21:31,50.850000


In [765]:
final_df4=pd.concat([final_df3,final_df4])
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,25.642470,19.434197,2022-11-21 07:30:39,2022-11-21 08:22:25,51.766667
1,26.253259,19.245808,2022-11-21 08:31:51,2022-11-21 09:20:31,48.666667
2,28.286558,19.309145,2022-11-21 09:30:06,2022-11-21 10:21:01,50.916667
3,29.738817,19.373728,2022-11-21 10:31:04,2022-11-21 14:23:31,232.450000
4,27.200955,19.348746,2022-11-21 14:30:26,2022-11-21 15:28:11,57.750000
...,...,...,...,...,...
14,27.893250,19.063268,2022-11-03 10:31:26,2022-11-03 11:23:33,52.116667
15,25.868228,18.922126,2022-11-03 11:29:52,2022-11-03 12:21:51,51.983333
16,0.600619,0.133253,2022-11-03 14:21:56,2022-11-03 14:22:03,0.116667
17,0.218496,0.138214,2022-11-03 15:28:56,2022-11-03 15:29:05,0.150000


In [766]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 80)& (rt["duration(min)"]>40)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='7'
final_df4['stop_id']='326'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/865668587.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='7'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/865668587.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='326'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,25.642470,19.434197,2022-11-21 07:30:39,2022-11-21 08:22:25,51.766667,7,326
1,26.253259,19.245808,2022-11-21 08:31:51,2022-11-21 09:20:31,48.666667,7,326
2,28.286558,19.309145,2022-11-21 09:30:06,2022-11-21 10:21:01,50.916667,7,326
4,27.200955,19.348746,2022-11-21 14:30:26,2022-11-21 15:28:11,57.750000,7,326
5,27.120204,18.876091,2022-11-21 15:31:53,2022-11-21 16:22:55,51.033333,7,326
...,...,...,...,...,...,...,...
12,29.754572,19.126723,2022-11-03 08:29:32,2022-11-03 09:26:41,57.150000,7,326
13,27.784271,19.141227,2022-11-03 09:29:39,2022-11-03 10:22:05,52.433333,7,326
14,27.893250,19.063268,2022-11-03 10:31:26,2022-11-03 11:23:33,52.116667,7,326
15,25.868228,18.922126,2022-11-03 11:29:52,2022-11-03 12:21:51,51.983333,7,326


In [767]:
final_df2.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt7new.csv",index=False,sep=',')

# rt 8

In [768]:
df1=pd.read_csv("rt8.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1902    41758
1808    34935
1807     5465
1811     3691
1806     1085
2113      430
2004      195
Name: vid, dtype: int64

In [804]:
df2=df3[df3['vid']==1902]
df2=df2[7000:8000]

In [805]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="409"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [799]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,33.739435,29.116586,2022-11-03 15:48:59,2022-11-03 17:33:05,104.100000
1,inf,30.915272,2022-11-03 17:34:39,2022-11-03 06:42:14,-652.416667
2,32.967165,29.247405,2022-11-03 06:49:03,2022-11-03 08:13:17,84.233333
3,38.819512,29.539298,2022-11-03 08:21:14,2022-11-03 09:41:14,80.000000
4,35.170539,29.208411,2022-11-03 09:54:18,2022-11-03 11:10:35,76.283333
...,...,...,...,...,...
3,57.553046,46.445018,2022-11-07 15:02:44,2022-11-07 07:27:23,-455.350000
4,35.980534,29.232203,2022-11-07 07:32:27,2022-11-07 08:51:05,78.633333
5,33.431906,29.244956,2022-11-07 09:02:15,2022-11-07 10:25:23,83.133333
6,35.288189,29.034938,2022-11-14 07:32:14,2022-11-14 08:55:40,83.433333


In [806]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 90)& (rt["duration(min)"]>60)&(rt["energy(kwh)"] < 40)&(rt["energy(kwh)"] > 20)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='8'
final_df4['stop_id']='409'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/3522010100.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='8'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/3522010100.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='409'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
2,32.967165,29.247405,2022-11-03 06:49:03,2022-11-03 08:13:17,84.233333,8,409
3,38.819512,29.539298,2022-11-03 08:21:14,2022-11-03 09:41:14,80.000000,8,409
4,35.170539,29.208411,2022-11-03 09:54:18,2022-11-03 11:10:35,76.283333,8,409
6,35.541644,28.943298,2022-11-03 19:06:31,2022-11-03 20:10:43,64.200000,8,409
7,31.194015,29.110649,2022-11-04 11:19:04,2022-11-04 12:44:31,85.450000,8,409
1,27.352195,27.151036,2022-11-01 18:27:07,2022-11-01 19:47:52,80.750000,8,409
2,32.501889,27.815813,2022-11-02 07:31:55,2022-11-02 08:58:44,86.816667,8,409
3,37.206176,29.716177,2022-11-02 09:03:45,2022-11-02 10:24:08,80.383333,8,409
6,37.370966,29.087680,2022-11-03 10:35:15,2022-11-03 11:59:37,84.366667,8,409
7,38.302309,29.203561,2022-11-04 12:02:17,2022-11-04 13:28:50,86.550000,8,409


In [807]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt8new.csv",index=False,sep=',')

# rt 9

In [268]:
df2=pd.read_csv("rt9.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1501    34790
2113    33355
2578    22496
2581    14040
2574     4846
701      3166
712      2949
702      2806
1811     2797
1302     2683
803      2021
1803     1925
577      1771
1001     1739
1104     1687
1200     1578
2571     1229
705       861
1805      788
576       694
1206      480
1601      463
1008      396
1401      331
2575      258
2002      176
1101      133
1017        4
Name: vid, dtype: int64

In [265]:
df2=df3[df3['vid']==2574]

df2

Unnamed: 0     vid             tmstmp        lat        lon    hdg  \
vid                                                                             
2574 0       172173.0  2574.0  20221103 13:37:46  29.619478 -82.362830  355.0   
     1       172232.0  2574.0  20221103 13:38:08  29.619733 -82.362851  355.0   
     2       172292.0  2574.0  20221103 13:38:16  29.619801 -82.361830   85.0   
     3       172350.0  2574.0  20221103 13:38:42  29.620013 -82.359147   84.0   
     4       172409.0  2574.0  20221103 13:38:57  29.620158 -82.357315   85.0   
...               ...     ...                ...        ...        ...    ...   
     4841   3398065.0  2574.0  20221205 18:44:23  29.623673 -82.356079  176.0   
     4842   3398104.0  2574.0  20221205 18:44:39  29.622198 -82.355945  173.0   
     4843   3398143.0  2574.0  20221205 18:44:50  29.620921 -82.355825  174.0   
     4844   3398182.0  2574.0  20221205 18:45:06  29.620759 -82.355863  114.0   
     4845   3398221.0  2574.0  20221205 18:45:25  29.619896 -82.355773  171.0   

                pid   rt                    des    pdist  ...        Froll  \
vid                                                       ...                
2574 0     500433.0  9.0  Reitz Union/UF Campus   3313.0  ...  1210.505685   
     1     500433.0  9.0  Reitz Union/UF Campus   3425.0  ...  1197.699118   
     2     500433.0  9.0  Reitz Union/UF Campus   3750.0  ...  1177.200000   
     3     500433.0  9.0  Reitz Union/UF Campus   4605.0  ...  1232.136000   
     4     500433.0  9.0  Reitz Union/UF Campus   5187.0  ...  1218.402000   
...             ...  ...                    ...      ...  ...          ...   
     4841  500432.0  9.0            Hunters Run  12872.0  ...  1183.262012   
     4842  500432.0  9.0            Hunters Run  13413.0  ...  1184.067000   
     4843  500432.0  9.0            Hunters Run  13879.0  ...  1177.200000   
     4844  500432.0  9.0            Hunters Run  13970.0  ...  1190.934000   
     4845  500432.0  9.0            Hunters Run  14287.0  ...          NaN   

                Fclimb       Fdrag     Finertia          Ftr         n  \
vid                                                                      
2574 0     4993.037865         NaN          NaN          NaN       NaN   
     1     1562.232842    4.492613  1364.589600  4129.014174  1.192513   
     2        0.000000  413.431036  2950.464000  4541.095036  1.192513   
     3        0.000000  270.883030     0.000000  1503.019030  1.192513   
     4        0.000000  379.414865   407.164032  2004.980897  1.192513   
...                ...         ...          ...          ...       ...   
     4841  4365.399203    0.597739  3956.900053  9506.159007  1.192513   
     4842     0.000000  285.662306  7419.187600  8888.916906  1.192513   
     4843     0.000000  453.066716 -7510.272000 -5880.005284  0.503139   
     4844     0.000000    3.550960 -5969.772160 -4775.287200  0.503139   
     4845          NaN   69.447757  4740.153347          NaN       NaN   

                Etr       Etr_a+       Etr_a0       Etr_a-  
vid                                                         
2574 0          NaN          NaN          NaN          NaN  
     1     0.038773  4124.529346  2764.424574  4124.513776  
     2     0.135357  4127.723743  1590.631036  4127.604257  
     3      0.12966  1232.136000  1503.019030  1232.136000  
     4     0.095749  1625.580340  1597.816865  1625.551724  
...             ...          ...          ...          ...  
     4841  0.022223  9505.566947  5549.258953  9505.555589  
     4842   0.46903  8603.502905  1469.729306  8603.006295  
     4843 -0.126118 -6333.290896  1630.266716 -6332.853104  
     4844 -0.012255 -4778.860307  1194.484960 -4778.816013  
     4845       NaN          NaN          NaN          NaN  

[4846 rows x 42 columns]

In [264]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="473"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [204]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,16.541415,12.546934,2022-11-07 13:52:15,2022-11-07 14:24:14,31.983333
1,84.376664,62.740289,2022-11-07 14:33:41,2022-11-07 08:36:57,-356.733333
2,19.370974,12.926502,2022-11-07 08:39:59,2022-11-07 09:30:42,50.716667
3,35.578893,25.183354,2022-11-07 09:32:50,2022-11-07 10:47:54,75.066667
4,0.531601,0.121187,2022-11-08 07:51:18,2022-11-08 07:51:22,0.066667
5,16.665242,12.686096,2022-11-09 10:07:10,2022-11-09 10:47:38,40.466667
6,37.527771,25.182410,2022-11-09 10:50:03,2022-11-09 12:14:31,84.466667
7,18.733016,12.560213,2022-11-15 07:54:27,2022-11-15 08:36:10,41.716667
8,0.087925,0.017019,2022-11-15 08:37:57,2022-11-15 08:37:57,0.000000
9,17.917824,12.823173,2022-11-15 08:43:00,2022-11-15 09:22:52,39.866667


In [224]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 50)& (rt["duration(min)"]>20)&(rt["energy(kwh)"] < 50)&(rt["energy(kwh)"] > 15)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,16.541415,12.546934,2022-11-07 13:52:15,2022-11-07 14:24:14,31.983333
5,16.665242,12.686096,2022-11-09 10:07:10,2022-11-09 10:47:38,40.466667
7,18.733016,12.560213,2022-11-15 07:54:27,2022-11-15 08:36:10,41.716667
9,17.917824,12.823173,2022-11-15 08:43:00,2022-11-15 09:22:52,39.866667
2,19.254998,12.623819,2022-12-01 08:19:13,2022-12-01 08:59:45,40.533333
3,17.073559,12.603691,2022-12-01 09:03:38,2022-12-01 09:45:20,41.700000
1,16.689350,12.610497,2022-11-29 10:18:49,2022-11-29 10:57:36,38.783333
0,17.325706,12.613720,2022-11-22 17:44:55,2022-11-22 18:17:18,32.383333
1,16.529291,12.740387,2022-11-22 18:22:45,2022-11-22 18:59:42,36.950000
2,17.344943,12.751891,2022-11-14 15:07:12,2022-11-14 15:54:06,46.900000


In [62]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 50)& (rt["duration(min)"]>20)&(rt["energy(kwh)"] < 40)&(rt["energy(kwh)"] > 15)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='9'
final_df4['stop_id']='447'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_18528/3492122922.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='9'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18528/3492122922.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='447'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
1,17.445765,12.616017,2022-11-16 08:47:48,2022-11-16 09:29:42,41.900000,9,447
2,16.929995,12.549783,2022-11-16 09:31:16,2022-11-16 10:14:27,43.183333,9,447
3,17.434507,12.542773,2022-11-16 10:16:16,2022-11-16 10:57:39,41.383333,9,447
0,16.721074,12.736829,2022-11-22 07:44:44,2022-11-22 08:20:23,35.650000,9,447
1,16.525758,12.794132,2022-11-22 08:30:12,2022-11-22 09:11:24,41.200000,9,447
2,16.447924,12.815111,2022-11-22 09:13:06,2022-11-22 09:46:28,33.366667,9,447
3,17.358392,12.836901,2022-11-22 09:51:05,2022-11-22 10:34:42,43.616667,9,447
5,17.689909,12.547173,2022-11-30 07:12:53,2022-11-30 07:44:29,31.600000,9,447
6,18.261875,12.502505,2022-11-30 07:53:14,2022-11-30 08:31:16,38.033333,9,447
7,21.085397,12.573192,2022-11-30 08:40:19,2022-11-30 09:12:39,32.333333,9,447


In [41]:
final_df3.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt9_new.csv",index=False,sep=',')

In [487]:
final_df7=final_df2

In [488]:
final_df7

,energy(kwh),dist(km),start,end,duration(min)
0,16.558453,12.646719,2022-12-05 08:09:53,2022-12-05 08:54:32,44.650000
1,36.235504,25.502294,2022-12-05 08:59:25,2022-12-05 10:16:03,76.633333


# rt 10

In [6]:
df2=pd.read_csv("rt10.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

802     35125
703     26982
705      4560
1001     2027
803      1993
702      1420
2106     1119
701       817
2581      754
1808      191
1006       99
Name: vid, dtype: int64

In [65]:
df2=df3[df3['vid']==703]
df2=df2[7000:8000]
#df2=pd.concat([df3,df4,df5,df6,df7,df8])

In [66]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="520"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [60]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,41.802726,27.213075,2022-11-07 12:48:47,2022-11-07 13:50:17,61.500000
1,39.968176,27.171613,2022-11-07 13:55:20,2022-11-07 15:08:30,73.166667
0,42.901035,26.991644,2022-11-04 12:45:07,2022-11-04 13:57:30,72.383333
0,43.097577,27.212011,2022-11-03 14:01:37,2022-11-03 15:04:32,62.916667
1,43.996255,27.199185,2022-11-03 15:13:54,2022-11-03 16:19:38,65.733333
2,42.805233,27.131197,2022-11-03 16:21:51,2022-11-03 17:29:05,67.233333
3,40.827264,27.303468,2022-12-02 08:05:13,2022-12-02 09:11:33,66.333333
4,44.301663,27.216524,2022-12-02 09:17:56,2022-12-02 10:21:41,63.750000
5,114.877848,74.493465,2022-12-02 10:27:54,2022-12-02 16:19:59,352.083333
6,41.052486,27.192824,2022-12-05 08:08:58,2022-12-05 09:15:34,66.600000


In [67]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 70)& (rt["duration(min)"]>20)&(rt["energy(kwh)"] < 55)&(rt["energy(kwh)"] > 15)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='10'
final_df4['stop_id']='520'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/1389758090.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='10'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/1389758090.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='520'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,41.802726,27.213075,2022-11-07 12:48:47,2022-11-07 13:50:17,61.500000,10,520
0,43.097577,27.212011,2022-11-03 14:01:37,2022-11-03 15:04:32,62.916667,10,520
1,43.996255,27.199185,2022-11-03 15:13:54,2022-11-03 16:19:38,65.733333,10,520
2,42.805233,27.131197,2022-11-03 16:21:51,2022-11-03 17:29:05,67.233333,10,520
3,40.827264,27.303468,2022-12-02 08:05:13,2022-12-02 09:11:33,66.333333,10,520
4,44.301663,27.216524,2022-12-02 09:17:56,2022-12-02 10:21:41,63.750000,10,520
6,41.052486,27.192824,2022-12-05 08:08:58,2022-12-05 09:15:34,66.600000,10,520
0,42.334462,27.110237,2022-11-07 12:11:55,2022-11-07 13:13:01,61.100000,10,520
1,40.767450,27.291615,2022-11-07 13:21:58,2022-11-07 14:24:33,62.583333,10,520
1,42.283561,27.224959,2022-12-07 08:40:35,2022-12-07 09:47:33,66.966667,10,520


In [68]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt10new.csv",index=False,sep=',')

# rt 11

In [69]:
df2=pd.read_csv("rt11.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2107    25077
710     20176
2111     6787
1811     6655
2005     2156
1806     2132
1101     1339
707      1130
702      1059
2201      943
1202      919
1003      875
712       873
1605      854
Name: vid, dtype: int64

In [107]:
df2=df3[df3['vid']==710]
df2
#df2=df2.head(2000)

Unnamed: 0    vid             tmstmp        lat        lon    hdg  \
vid                                                                            
710 0            15.0  710.0  20221101 14:09:30  29.659210 -82.294381  270.0   
    1            73.0  710.0  20221101 14:09:47  29.659210 -82.294575  270.0   
    2           133.0  710.0  20221101 14:10:07  29.659220 -82.297092  270.0   
    3           192.0  710.0  20221101 14:10:23  29.659220 -82.299583  270.0   
    4           252.0  710.0  20221101 14:10:31  29.659498 -82.299872  359.0   
...               ...    ...                ...        ...        ...    ...   
    20171   4386733.0  710.0  20221209 11:00:37  29.642070 -82.272395  270.0   
    20172   4386834.0  710.0  20221209 11:00:52  29.643471 -82.272460    0.0   
    20173   4386855.0  710.0  20221209 11:01:05  29.645864 -82.272460    0.0   
    20174   4386916.0  710.0  20221209 11:01:18  29.646170 -82.272438   89.0   
    20175   4386977.0  710.0  20221209 11:01:36  29.647744 -82.272460    0.0   

            pid    rt                  des    pdist  ...        Froll  \
vid                                                  ...                
710 0      44.0  11.0  Dowtown/Health Dept  17827.0  ...  1183.563690   
    1      44.0  11.0  Dowtown/Health Dept  17888.0  ...  1204.668000   
    2      44.0  11.0  Dowtown/Health Dept  18686.0  ...  1177.200000   
    3      44.0  11.0  Dowtown/Health Dept  19475.0  ...  1225.269000   
    4      44.0  11.0  Dowtown/Health Dept  19667.0  ...  1190.934000   
...         ...   ...                  ...      ...  ...          ...   
    20171  44.0  11.0  Dowtown/Health Dept    399.0  ...  1238.160664   
    20172  44.0  11.0  Dowtown/Health Dept    926.0  ...  1197.801000   
    20173  44.0  11.0  Dowtown/Health Dept   1799.0  ...  1196.885001   
    20174  44.0  11.0  Dowtown/Health Dept   1927.0  ...  1183.262012   
    20175  44.0  11.0  Dowtown/Health Dept   2483.0  ...          NaN   

                Fclimb        Fdrag      Finertia           Ftr         n  \
vid                                                                         
710 0      3452.030843          NaN           NaN           NaN       NaN   
    1         0.000000     3.259882  -7459.467219  -6251.539336  0.503139   
    2         0.000000   399.255119  12391.948800  13968.403919  1.192513   
    3         0.000000   611.224644  -7293.485540  -5456.991896  0.503139   
    4         0.000000    73.150576  -5223.550640  -3959.466064  0.503139   
...                ...          ...           ...           ...       ...   
    20171  4567.936365    15.242765  -4269.874620   1551.465174  1.192513   
    20172     0.000000   291.711383  14009.786560  15499.298943  1.192513   
    20173  4683.512866  1130.938940  -6004.194240   1007.142567  1.192513   
    20174  4365.399203    18.634013 -11870.700160  -6303.404932  0.503139   
    20175          NaN   255.231271  12624.980818           NaN       NaN   

                Etr        Etr_a+       Etr_a0        Etr_a-  
vid                                                           
710 0           NaN           NaN          NaN           NaN  
    1     -0.016329  -6254.827070  1207.927882  -6254.771367  
    2      1.093217  13569.765257  1576.455119  13568.532343  
    3     -0.204155  -6068.561590  1836.493644  -6067.871490  
    4     -0.023468  -4032.660618  1264.084576  -4032.572662  
...             ...           ...          ...           ...  
    20171  0.019346   1536.190863  5821.339794   1536.253956  
    20172  0.785452  15208.026670  1489.512383  15207.148450  
    20173 -0.010882   -124.117510  7011.336807   -123.475235  
    20174 -0.030175  -6322.121388  5567.295228  -6321.956502  
    20175       NaN           NaN          NaN           NaN  

[20176 rows x 42 columns]

In [108]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="326"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/3753238146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id'] = range(len(df2))
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/3753238146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id1'] = range(len(df2))


IndexError: list index out of range

In [106]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,20.047699,19.386853,2022-11-15 06:29:43,2022-11-15 07:20:02,50.316667
1,18.784441,19.487990,2022-11-15 07:29:49,2022-11-15 08:21:04,51.250000
2,22.128488,19.588617,2022-11-15 08:30:40,2022-11-15 09:19:30,48.833333
3,22.828463,19.502794,2022-11-15 09:29:43,2022-11-15 10:21:55,52.200000
4,20.326385,19.424343,2022-11-15 10:29:55,2022-11-15 14:21:01,231.100000
...,...,...,...,...,...
0,41.802726,27.213075,2022-11-07 12:48:47,2022-11-07 13:50:17,61.500000
1,39.968176,27.171613,2022-11-07 13:55:20,2022-11-07 15:08:30,73.166667
0,22.522737,19.821668,2022-11-22 10:59:43,2022-11-22 14:58:36,238.883333
1,21.523940,19.782440,2022-11-22 15:00:16,2022-11-22 15:50:28,50.200000


In [109]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 60)& (rt["duration(min)"]>30)&(rt["energy(kwh)"] < 30)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='11'
final_df4['stop_id']='326'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/1665445134.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='11'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/1665445134.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='326'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,20.047699,19.386853,2022-11-15 06:29:43,2022-11-15 07:20:02,50.316667,11,326
1,18.784441,19.487990,2022-11-15 07:29:49,2022-11-15 08:21:04,51.250000,11,326
2,22.128488,19.588617,2022-11-15 08:30:40,2022-11-15 09:19:30,48.833333,11,326
3,22.828463,19.502794,2022-11-15 09:29:43,2022-11-15 10:21:55,52.200000,11,326
5,23.458813,19.605864,2022-11-15 14:29:29,2022-11-15 15:21:51,52.366667,11,326
...,...,...,...,...,...,...,...
0,18.371065,19.515694,2022-11-18 16:38:11,2022-11-18 17:25:27,47.266667,11,326
1,18.403942,18.597005,2022-11-18 17:34:04,2022-11-18 18:19:48,45.733333,11,326
2,17.721385,17.367707,2022-11-18 18:32:28,2022-11-18 19:17:05,44.616667,11,326
1,21.523940,19.782440,2022-11-22 15:00:16,2022-11-22 15:50:28,50.200000,11,326


In [110]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt11new.csv",index=False,sep=',')

# rt 13

In [111]:
df2=pd.read_csv("rt13.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2108    37698
709     24392
712      1043
1810      770
1602      706
Name: vid, dtype: int64

In [180]:
df2=df3[df3['vid']==2108]

df2=df2[28000:29000]
#df2=pd.concat([df3,df4,df5,df6,df7,df8,df9,df10])

In [181]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1264"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df2=DataFrame(final_output2)#将字典转换成为数据框
final_df2

IndexError: positional indexers are out-of-bounds

In [1304]:
final_df4=pd.concat([final_df3,final_df2])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,18.178309,11.391891,2022-12-06 15:44:04,2022-12-06 16:22:41,38.616667
1,15.225420,11.440623,2022-12-06 16:30:54,2022-12-06 17:16:55,46.016667
0,13.101660,8.931680,2022-11-28 14:21:38,2022-11-28 14:56:15,34.616667
0,54.453266,30.993808,2022-11-01 14:25:35,2022-11-01 16:01:34,95.983333
1,19.136207,10.291362,2022-11-01 16:12:56,2022-11-01 16:47:02,34.100000
2,33.588001,19.510214,2022-11-02 17:20:11,2022-11-02 07:05:23,-614.800000
3,52.735177,31.905352,2022-11-02 07:08:43,2022-11-02 08:44:56,96.216667
4,11.263493,6.388815,2022-11-02 09:21:39,2022-11-02 09:45:26,23.783333
5,20.125058,10.170794,2022-11-02 09:56:59,2022-11-02 10:27:08,30.150000
6,16.129071,10.407769,2022-11-02 10:35:41,2022-11-02 11:01:08,25.450000


In [1305]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 50)& (rt["duration(min)"]>20)&(rt["energy(kwh)"] < 50)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='13'
final_df4['stop_id']='681'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_32368/3059625972.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='13'
C:\Users\zheng\AppData\Local\Temp/ipykernel_32368/3059625972.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='681'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,18.178309,11.391891,2022-12-06 15:44:04,2022-12-06 16:22:41,38.616667,13,681
1,15.225420,11.440623,2022-12-06 16:30:54,2022-12-06 17:16:55,46.016667,13,681
0,13.101660,8.931680,2022-11-28 14:21:38,2022-11-28 14:56:15,34.616667,13,681
1,19.136207,10.291362,2022-11-01 16:12:56,2022-11-01 16:47:02,34.100000,13,681
4,11.263493,6.388815,2022-11-02 09:21:39,2022-11-02 09:45:26,23.783333,13,681
5,20.125058,10.170794,2022-11-02 09:56:59,2022-11-02 10:27:08,30.150000,13,681
6,16.129071,10.407769,2022-11-02 10:35:41,2022-11-02 11:01:08,25.450000,13,681
9,16.875690,10.411912,2022-11-03 15:33:42,2022-11-03 16:08:09,34.450000,13,681
12,16.786146,10.575317,2022-11-03 19:32:56,2022-11-03 19:58:41,25.750000,13,681
13,13.356718,10.430535,2022-11-03 20:15:59,2022-11-03 20:39:00,23.016667,13,681


In [1306]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt13.csv",index=False,sep=',')

# rt 15

In [182]:
df2=pd.read_csv("rt15.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1809    40177
2109    37574
1807     3330
1804     2234
803      2188
1602     1916
2574      794
702       483
Name: vid, dtype: int64

In [304]:
df2=df3[df3['vid']==1809]
df2=df2[6000:7000]
#df2=pd.concat([df3,df4,df5])


In [305]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="255"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [195]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,47.521579,45.491268,2022-11-17 07:29:53,2022-11-17 09:29:01,119.133333
1,23.920323,22.711982,2022-11-17 09:31:06,2022-11-17 10:22:07,51.016667
2,33.498452,32.644094,2022-11-17 10:29:09,2022-11-17 14:45:32,256.383333
3,23.715654,22.755455,2022-11-17 14:50:07,2022-11-17 15:59:59,69.866667
0,25.849105,22.402862,2022-11-28 14:18:28,2022-11-28 15:29:13,70.750000


In [74]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 70)& (rt["duration(min)"]>25)&(rt["energy(kwh)"] < 40)&(rt["energy(kwh)"] > 15)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='15'
final_df4['stop_id']='1'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_35812/2861191222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='15'
C:\Users\zheng\AppData\Local\Temp/ipykernel_35812/2861191222.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='1'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
1,24.225124,22.679909,2022-12-07 09:04:19,2022-12-07 09:58:12,53.883333,15,1
3,22.341905,22.764455,2022-12-07 14:22:55,2022-12-07 15:28:44,65.816667,15,1
4,24.274909,22.488565,2022-12-07 15:33:49,2022-12-07 16:33:15,59.433333,15,1
5,22.819152,22.599101,2022-12-07 16:44:19,2022-12-07 17:42:25,58.100000,15,1
6,23.197782,22.573198,2022-12-07 18:06:21,2022-12-07 18:54:55,48.566667,15,1
...,...,...,...,...,...,...,...
66,26.258504,22.691822,2022-12-08 08:07:12,2022-12-08 08:54:54,47.700000,15,1
67,24.142737,22.797011,2022-12-08 09:00:05,2022-12-08 09:50:16,50.183333,15,1
69,23.543600,22.577206,2022-12-08 14:23:05,2022-12-08 15:23:54,60.816667,15,1
70,21.567267,22.589739,2022-12-08 15:30:37,2022-12-08 16:30:31,59.900000,15,1


In [75]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt15.csv",index=False,sep=',')

# rt 16

In [308]:
df1=pd.read_csv("rt16.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2110    38716
1811      820
710       671
1808      440
1008       14
Name: vid, dtype: int64

In [415]:
df2=df3[df3['vid']==2110]
df2=df2[37000:38000]

In [416]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="13"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

ValueError: cannot join with no overlapping index names

In [389]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,0.231007,0.128274,2022-11-22 17:10:29,2022-11-22 17:11:02,0.550000
1,17.191566,10.232663,2022-11-22 17:42:28,2022-11-22 18:14:56,32.466667
0,16.258712,10.575746,2022-11-21 17:13:32,2022-11-21 17:49:33,36.016667
0,0.091410,0.036005,2022-11-18 08:10:36,2022-11-18 08:10:36,0.000000
1,78.078402,48.891224,2022-11-18 08:12:26,2022-11-18 10:55:40,163.233333
2,16.788613,10.077579,2022-11-18 10:57:37,2022-11-18 11:29:28,31.850000
0,30.971374,18.212124,2022-11-14 14:28:46,2022-11-14 15:59:21,90.583333
1,15.143770,10.020927,2022-11-14 16:01:17,2022-11-14 16:34:12,32.916667
2,19.877817,10.447098,2022-11-14 16:36:00,2022-11-14 17:22:15,46.250000
0,19.618737,9.913279,2022-11-08 06:57:49,2022-11-08 07:34:05,36.266667


In [412]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 40)& (rt["duration(min)"]>25)&(rt["energy(kwh)"] < 25)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='16'
final_df4['stop_id']='13'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/12387383.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='16'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/12387383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='13'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
1,17.191566,10.232663,2022-11-22 17:42:28,2022-11-22 18:14:56,32.466667,16,13
0,16.258712,10.575746,2022-11-21 17:13:32,2022-11-21 17:49:33,36.016667,16,13
2,16.788613,10.077579,2022-11-18 10:57:37,2022-11-18 11:29:28,31.850000,16,13
1,15.143770,10.020927,2022-11-14 16:01:17,2022-11-14 16:34:12,32.916667,16,13
0,19.618737,9.913279,2022-11-08 06:57:49,2022-11-08 07:34:05,36.266667,16,13
0,16.900034,9.891933,2022-11-07 08:40:41,2022-11-07 09:13:55,33.233333,16,13
0,14.666247,10.139494,2022-11-04 07:33:06,2022-11-04 08:08:02,34.933333,16,13


In [417]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt16new.csv",index=False,sep=',')

# rt 17

In [446]:
df2=pd.read_csv("rt17.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2102    44080
1601      155
Name: vid, dtype: int64

In [522]:
df2=df3[df3['vid']==2102]
df2=df2[43000:44000]

In [523]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="118"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

ValueError: cannot join with no overlapping index names

In [519]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,11.551186,7.412201,2022-12-08 07:51:17,2022-12-08 08:25:45,34.466667
1,44.476707,29.298922,2022-12-08 08:29:17,2022-12-08 10:37:15,127.966667
0,8.832535,6.807006,2022-12-07 08:23:42,2022-12-07 08:58:09,34.450000
1,10.861474,7.403972,2022-12-07 09:31:14,2022-12-07 10:04:54,33.666667
2,32.757501,20.909485,2022-12-07 10:06:32,2022-12-07 14:37:51,271.316667
0,0.095820,0.035357,2022-11-18 19:12:27,2022-11-18 19:12:27,0.000000
1,21.079182,15.239986,2022-11-21 06:44:40,2022-11-21 07:49:04,64.400000
2,11.731434,7.540004,2022-11-21 07:50:39,2022-11-21 08:21:39,31.000000
3,21.040363,15.163020,2022-11-21 08:58:47,2022-11-21 10:05:02,66.250000
4,21.692926,14.071607,2022-11-21 10:07:05,2022-11-21 14:05:26,238.350000


In [524]:
def clean_df(rt):
    rt_filtered = rt[(rt["dist(km)"] < 10)& (rt["dist(km)"]>6)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='17'
final_df4['stop_id']='118'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/3733653251.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='17'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/3733653251.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='118'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,11.551186,7.412201,2022-12-08 07:51:17,2022-12-08 08:25:45,34.466667,17,118
0,8.832535,6.807006,2022-12-07 08:23:42,2022-12-07 08:58:09,34.450000,17,118
1,10.861474,7.403972,2022-12-07 09:31:14,2022-12-07 10:04:54,33.666667,17,118
2,11.731434,7.540004,2022-11-21 07:50:39,2022-11-21 08:21:39,31.000000,17,118
6,10.170056,7.206313,2022-11-21 15:48:29,2022-11-21 16:23:50,35.350000,17,118
0,10.551875,7.472930,2022-11-16 16:20:22,2022-11-16 16:52:52,32.500000,17,118
1,12.820169,7.408082,2022-11-15 09:32:03,2022-11-15 10:14:22,42.316667,17,118
2,10.368545,7.097226,2022-11-15 10:15:47,2022-11-15 10:45:20,29.550000,17,118
1,13.045137,9.643612,2022-11-07 15:51:15,2022-11-07 06:42:23,-548.866667,17,118
1,12.977129,7.408014,2022-11-01 16:20:35,2022-11-01 16:52:49,32.233333,17,118


In [525]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt17new.csv",index=False,sep=',')

# rt 20

In [5]:
df2=pd.read_csv("rt20.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1102    40028
1604    33027
1103    28134
1607    27415
1602     9202
707      5748
1013     5106
1016     1887
1106     1556
576      1400
1206     1090
1501      683
1806      420
1811      137
1017       46
Name: vid, dtype: int64

In [120]:
df2=df3[df3['vid']==1501]
df2=df2.head(1000)
#df2=df2[2000:6000]
#df2=df2[12000:13000]
#df2=pd.concat([df4,df5,df6,df7,df8])

In [121]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="171"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [119]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,60.443320,40.073348,2022-11-28 12:33:21,2022-11-28 14:32:19,118.966667
1,31.228683,21.523441,2022-11-28 14:34:08,2022-11-28 15:30:18,56.166667
0,30.149235,20.448001,2022-11-01 15:00:49,2022-11-01 16:03:12,62.383333
0,inf,20.531193,2022-12-07 07:15:19,2022-12-07 08:08:05,52.766667
1,28.201846,20.763198,2022-12-07 08:15:08,2022-12-07 09:10:14,55.100000
2,1.534075,1.039308,2022-12-07 10:04:05,2022-12-07 10:06:18,2.216667
3,32.814575,21.769738,2022-12-07 15:24:53,2022-12-07 16:30:06,65.216667
4,29.636745,20.765480,2022-12-07 16:33:20,2022-12-07 17:48:16,74.933333
0,27.878526,20.540716,2022-11-04 06:35:03,2022-11-04 07:25:17,50.233333
1,28.929939,20.617492,2022-11-04 07:33:37,2022-11-04 08:27:19,53.700000


In [122]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 70)& (rt["duration(min)"]>50)&(rt["energy(kwh)"] < 40)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='20'
final_df4['stop_id']='171'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/143551052.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='20'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/143551052.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='171'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
1,31.228683,21.523441,2022-11-28 14:34:08,2022-11-28 15:30:18,56.166667,20,171
0,30.149235,20.448001,2022-11-01 15:00:49,2022-11-01 16:03:12,62.383333,20,171
1,28.201846,20.763198,2022-12-07 08:15:08,2022-12-07 09:10:14,55.100000,20,171
3,32.814575,21.769738,2022-12-07 15:24:53,2022-12-07 16:30:06,65.216667,20,171
0,27.878526,20.540716,2022-11-04 06:35:03,2022-11-04 07:25:17,50.233333,20,171
1,28.929939,20.617492,2022-11-04 07:33:37,2022-11-04 08:27:19,53.700000,20,171
2,29.445210,20.955422,2022-11-04 08:32:50,2022-11-04 09:29:00,56.166667,20,171
3,31.327075,20.467300,2022-11-04 09:33:51,2022-11-04 10:29:57,56.100000,20,171
4,29.682656,21.010023,2022-11-15 06:44:23,2022-11-15 07:36:44,52.350000,20,171
6,28.523004,21.053329,2022-11-15 09:44:17,2022-11-15 10:37:34,53.283333,20,171


In [123]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt20_new.csv",index=False,sep=',')

# rt 23

In [526]:
df2=pd.read_csv("rt23.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1002    34367
1402    23559
1013     4170
1012     2348
1008     1741
705      1660
1001     1643
576      1519
571      1096
1602     1069
1302     1007
1103      658
Name: vid, dtype: int64

In [648]:
df2=df3[df3['vid']==1002]
df2=df2[1000:2000]

In [649]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="175"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [643]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,45.878630,26.316160,2022-12-08 08:55:22,2022-12-08 09:59:18,63.933333
1,24.666825,13.125104,2022-12-08 10:02:14,2022-12-08 10:35:25,33.183333
2,22.901995,12.658788,2022-12-08 10:38:10,2022-12-08 14:04:38,206.466667
3,25.204806,13.055641,2022-12-08 14:10:20,2022-12-08 14:47:47,37.450000
4,20.754936,13.128803,2022-12-08 14:54:18,2022-12-08 15:36:15,41.950000
...,...,...,...,...,...
0,0.000000,0.000000,2022-11-22 08:31:00,2022-11-22 08:31:00,0.000000
1,24.029464,12.799802,2022-11-22 08:32:41,2022-11-22 09:10:59,38.300000
2,22.924585,13.076910,2022-11-22 09:13:53,2022-11-22 09:45:39,31.766667
3,23.253063,13.294553,2022-11-22 10:24:11,2022-11-22 10:59:08,34.950000


In [650]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 45)& (rt["duration(min)"]>25)&(rt["energy(kwh)"] < 35)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='23'
final_df4['stop_id']='175'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/1050468016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='23'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/1050468016.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='175'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
1,24.666825,13.125104,2022-12-08 10:02:14,2022-12-08 10:35:25,33.183333,23,175
3,25.204806,13.055641,2022-12-08 14:10:20,2022-12-08 14:47:47,37.450000,23,175
4,20.754936,13.128803,2022-12-08 14:54:18,2022-12-08 15:36:15,41.950000,23,175
0,21.993575,12.924640,2022-11-28 13:26:20,2022-11-28 14:04:30,38.166667,23,175
1,24.356007,13.186944,2022-11-28 14:10:20,2022-11-28 14:49:39,39.316667,23,175
3,24.102463,13.138385,2022-11-30 07:27:20,2022-11-30 08:05:52,38.533333,23,175
0,21.739034,13.149902,2022-11-22 08:50:07,2022-11-22 09:22:27,32.333333,23,175
1,24.025010,13.022943,2022-11-22 09:26:20,2022-11-22 09:59:33,33.216667,23,175
2,20.886399,13.151295,2022-11-22 10:02:11,2022-11-22 10:34:54,32.716667,23,175
4,20.659084,13.021550,2022-11-22 14:10:05,2022-11-22 14:49:09,39.066667,23,175


In [651]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt23new.csv",index=False,sep=',')

# rt 25

In [653]:
df1=pd.read_csv("rt25.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2582.0    10211
705.0      4641
1206.0     3207
707.0      2634
702.0      1956
1806.0     1919
803.0      1728
1604.0     1673
1007.0     1653
701.0      1644
712.0      1466
572.0      1374
1016.0     1336
Name: vid, dtype: int64

In [732]:
df2=df3[df3['vid']==2582]
df2=df2[9000:10000]

In [733]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="473"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [725]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,30.960875,24.481206,2022-12-05 15:01:12,2022-12-05 16:13:37,72.416667
1,26.136072,24.355935,2022-12-05 16:16:37,2022-12-05 17:34:45,78.133333
2,0.000000,0.000000,2022-12-05 17:37:11,2022-12-05 17:37:11,0.000000
0,0.063947,0.013899,2022-11-22 08:20:23,2022-11-22 08:20:23,0.000000
1,26.359634,22.867200,2022-11-22 08:25:42,2022-11-22 09:25:59,60.283333
2,0.351702,0.102068,2022-11-22 10:31:58,2022-11-22 10:31:58,0.000000
3,44.721190,37.425977,2022-11-22 10:35:12,2022-11-22 14:52:54,257.700000
0,26.245668,23.048196,2022-11-04 14:54:29,2022-11-04 15:59:40,65.183333
0,26.503970,22.996289,2022-11-09 09:26:11,2022-11-09 10:30:29,64.300000
0,23.644049,22.498136,2022-11-03 13:43:14,2022-11-03 14:48:48,65.566667


In [734]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 80)& (rt["duration(min)"]>50)&(rt["energy(kwh)"] > 15)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='25'
final_df4['stop_id']='473'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/2260192048.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='25'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/2260192048.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='473'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,30.960875,24.481206,2022-12-05 15:01:12,2022-12-05 16:13:37,72.416667,25,473
1,26.136072,24.355935,2022-12-05 16:16:37,2022-12-05 17:34:45,78.133333,25,473
1,26.359634,22.867200,2022-11-22 08:25:42,2022-11-22 09:25:59,60.283333,25,473
0,26.245668,23.048196,2022-11-04 14:54:29,2022-11-04 15:59:40,65.183333,25,473
0,26.503970,22.996289,2022-11-09 09:26:11,2022-11-09 10:30:29,64.300000,25,473
0,23.644049,22.498136,2022-11-03 13:43:14,2022-11-03 14:48:48,65.566667,25,473
1,26.560574,23.022619,2022-11-03 14:56:56,2022-11-03 15:56:36,59.666667,25,473
1,27.412736,23.013872,2022-11-18 09:31:27,2022-11-18 10:30:12,58.750000,25,473
2,27.983383,23.067249,2022-11-18 10:34:55,2022-11-18 11:31:46,56.850000,25,473
0,26.485775,23.047050,2022-11-17 16:03:41,2022-11-17 17:06:02,62.350000,25,473


In [735]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt25new.csv",index=False,sep=',')

# rt 26

In [736]:
df1=pd.read_csv("rt26.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2106.0    40321
2111.0     1425
2114.0       25
Name: vid, dtype: int64

In [828]:
df2=df3[df3['vid']==2106]
df2=df2[33000:34000]

In [829]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [827]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,28.115473,25.441048,2022-11-30 07:00:54,2022-11-30 07:57:42,56.800000
1,27.438613,25.405515,2022-11-30 08:00:21,2022-11-30 08:53:11,52.833333
2,30.905153,25.496245,2022-11-30 09:01:58,2022-11-30 09:51:05,49.116667
3,26.316845,25.276235,2022-11-30 10:00:21,2022-11-30 10:54:51,54.500000
0,27.279479,25.350080,2022-11-29 08:01:39,2022-11-29 08:55:22,53.716667
...,...,...,...,...,...
1,27.527328,25.329703,2022-11-01 16:04:28,2022-11-01 16:57:24,52.933333
2,26.789545,25.251602,2022-11-01 17:04:30,2022-11-01 17:58:16,53.766667
3,28.994015,22.310588,2022-11-01 18:01:56,2022-11-01 18:53:13,51.283333
4,17.367663,17.977768,2022-11-01 19:05:06,2022-11-01 19:56:59,51.883333


In [830]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 80)& (rt["duration(min)"]>50)&(rt["energy(kwh)"] > 15)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='26'
final_df4['stop_id']='1'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/2454325243.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='26'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/2454325243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='1'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,28.115473,25.441048,2022-11-30 07:00:54,2022-11-30 07:57:42,56.800000,26,1
1,27.438613,25.405515,2022-11-30 08:00:21,2022-11-30 08:53:11,52.833333,26,1
3,26.316845,25.276235,2022-11-30 10:00:21,2022-11-30 10:54:51,54.500000,26,1
0,27.279479,25.350080,2022-11-29 08:01:39,2022-11-29 08:55:22,53.716667,26,1
1,26.186193,25.423590,2022-11-29 09:00:05,2022-11-29 09:52:24,52.316667,26,1
2,26.555517,25.301431,2022-11-29 10:00:05,2022-11-29 10:51:48,51.716667,26,1
3,25.378984,25.428948,2022-11-29 14:02:31,2022-11-29 14:54:10,51.650000,26,1
1,29.469845,25.425282,2022-11-28 14:04:15,2022-11-28 15:08:37,64.366667,26,1
2,29.363396,25.271269,2022-11-28 15:10:15,2022-11-28 16:10:35,60.333333,26,1
1,26.504026,24.913404,2022-11-28 12:00:22,2022-11-28 12:54:23,54.016667,26,1


In [831]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt26new.csv",index=False,sep=',')

# rt 33

In [6]:
df1=pd.read_csv("rt33.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1202    33571
1601    29773
1606    29429
1017    14041
1602     5867
1807     2847
1013     2039
1001     1609
1401     1499
712       996
2577      800
1604      745
1206      668
1016      610
1008      578
2103      164
1012       94
Name: vid, dtype: int64

In [41]:
df2=df3[df3['vid']==1202]
df2=df2[24000:25000]
#df2=pd.concat([df9,df4,df5,df6,df7])

In [42]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="925"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [952]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,0.002621,0.003066,2022-11-02 10:16:52,2022-11-02 10:16:52,0.000000,NaN,NaN
1,23.574489,15.588147,2022-11-02 10:19:12,2022-11-02 11:26:34,67.366667,NaN,NaN
2,0.004085,0.004835,2022-11-02 11:28:25,2022-11-02 11:28:25,0.000000,NaN,NaN
3,23.158929,15.567285,2022-11-03 13:54:21,2022-11-03 14:56:19,61.966667,NaN,NaN
0,20.727555,15.372725,2022-11-02 15:06:29,2022-11-02 16:23:43,77.233333,NaN,NaN
1,32.432018,25.250478,2022-11-02 16:26:15,2022-11-02 07:50:41,-515.566667,NaN,NaN
2,23.155763,15.599344,2022-11-02 07:54:25,2022-11-02 09:13:54,79.483333,NaN,NaN
0,20.158343,15.697225,2022-12-09 08:37:25,2022-12-09 09:38:13,60.800000,33,925
2,20.610360,15.676517,2022-12-09 09:49:21,2022-12-09 10:57:53,68.533333,33,925
0,20.883957,15.464595,2022-12-09 09:28:48,2022-12-09 10:27:30,58.700000,33,925


In [945]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 77)& (rt["duration(min)"]>20)&(rt["energy(kwh)"] < 40)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='33'
final_df4['stop_id']='925'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/3221695316.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='33'
C:\Users\zheng\AppData\Local\Temp/ipykernel_23600/3221695316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='925'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,20.158343,15.697225,2022-12-09 08:37:25,2022-12-09 09:38:13,60.800000,33,925
2,20.610360,15.676517,2022-12-09 09:49:21,2022-12-09 10:57:53,68.533333,33,925
0,20.883957,15.464595,2022-12-09 09:28:48,2022-12-09 10:27:30,58.700000,33,925
0,20.508319,15.803156,2022-12-08 08:18:17,2022-12-08 09:17:36,59.316667,33,925
1,22.008872,15.358291,2022-12-08 09:28:32,2022-12-08 10:33:34,65.033333,33,925
1,20.062261,15.861996,2022-12-06 14:12:42,2022-12-06 15:18:17,65.583333,33,925
0,24.806278,15.561468,2022-12-05 08:12:41,2022-12-05 09:19:11,66.500000,33,925
0,20.741360,15.367214,2022-11-29 09:25:29,2022-11-29 10:31:01,65.533333,33,925
1,20.780037,15.399103,2022-11-17 14:12:59,2022-11-17 15:18:22,65.383333,33,925
0,22.488576,15.541305,2022-11-16 09:32:34,2022-11-16 10:37:29,64.916667,33,925


In [946]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt33new.csv",index=False,sep=',')

# rt 34

In [6]:
df2=pd.read_csv("rt34.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1014    41537
1804    29991
1803     3065
1007     2051
1013     1942
1016     1879
1809     1859
1806     1077
2574     1048
1801      411
2572      234
1202      134
Name: vid, dtype: int64

In [86]:
df2=df3[df3['vid']==1014]
df2=df2[23000:35000]
#df2=pd.concat([df9,df4,df5,df6,df7,df8])

In [87]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="951"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

KeyError: 'O'

In [85]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,23.989410,16.516779,2022-12-02 09:16:10,2022-12-02 10:04:52,48.700000
1,74.270248,48.921392,2022-12-02 10:14:50,2022-12-02 16:08:54,354.066667
2,25.757744,16.456478,2022-12-02 16:16:29,2022-12-02 17:17:06,60.616667
3,26.412693,16.558517,2022-12-05 07:16:05,2022-12-05 08:09:11,53.100000
4,inf,16.583846,2022-12-05 08:16:25,2022-12-05 09:04:11,47.766667
...,...,...,...,...,...
2,24.641752,16.529388,2022-11-28 14:22:07,2022-11-28 15:13:31,51.400000
3,25.914877,16.685036,2022-11-28 15:19:54,2022-11-28 16:09:45,49.850000
0,50.070673,32.074529,2022-11-14 14:14:39,2022-11-14 15:56:34,101.916667
1,27.188101,16.468863,2022-11-14 16:09:32,2022-11-14 17:10:38,61.100000


In [88]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 70)& (rt["duration(min)"]>30)&(rt["energy(kwh)"] < 40)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='34'
final_df4['stop_id']='951'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_7892/1719307382.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='34'
C:\Users\zheng\AppData\Local\Temp/ipykernel_7892/1719307382.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='951'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,23.989410,16.516779,2022-12-02 09:16:10,2022-12-02 10:04:52,48.700000,34,951
2,25.757744,16.456478,2022-12-02 16:16:29,2022-12-02 17:17:06,60.616667,34,951
3,26.412693,16.558517,2022-12-05 07:16:05,2022-12-05 08:09:11,53.100000,34,951
5,26.884291,16.461911,2022-12-05 09:20:21,2022-12-05 10:10:07,49.766667,34,951
7,27.147974,16.504108,2022-12-05 14:18:15,2022-12-05 15:12:57,54.700000,34,951
...,...,...,...,...,...,...,...
1,22.579704,15.294024,2022-11-28 13:26:16,2022-11-28 14:15:53,49.616667,34,951
2,24.641752,16.529388,2022-11-28 14:22:07,2022-11-28 15:13:31,51.400000,34,951
3,25.914877,16.685036,2022-11-28 15:19:54,2022-11-28 16:09:45,49.850000,34,951
1,27.188101,16.468863,2022-11-14 16:09:32,2022-11-14 17:10:38,61.100000,34,951


In [89]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt34new.csv",index=False,sep=',')

# rt 35

In [124]:
df2=pd.read_csv("rt35_.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1009    31696
1003    31312
1010    28919
1007    14780
1001    10121
1008     9251
1011     5359
1602     4074
1012     3624
705      2920
1013     1988
803      1957
1002     1471
702      1103
707      1100
701       863
1603      726
1501      611
1017      559
1403      526
576       519
1807      299
2004      146
Name: vid, dtype: int64

In [389]:
df2=df3[df3['vid']==1403]
#df2=df2[0:1000]
#df2
#df4=df2[8000:9000]
#df5=df2[12000:15000]
#df2=pd.concat([df4,df5])

In [390]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="473"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/1671898765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id'] = range(len(df2))
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/1671898765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id1'] = range(len(df2))


,energy(kwh),dist(km),start,end,duration(min)
0,0.326333,0.088768,2022-12-02 17:48:56,2022-12-02 17:49:12,0.266667


In [388]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,23.166649,15.737845,2022-11-29 16:22:41,2022-11-29 17:31:18,68.616667
0,13.157383,7.982764,2022-11-01 17:34:46,2022-11-01 18:08:58,34.200000
1,0.000000,0.000000,2022-11-01 18:32:40,2022-11-01 18:32:40,0.000000
2,0.562023,0.159695,2022-11-01 20:14:26,2022-11-01 20:14:31,0.083333
3,22.103195,15.981981,2022-11-01 20:31:28,2022-11-01 21:12:23,40.916667
...,...,...,...,...,...
27,24.183688,16.100508,2022-11-16 07:49:20,2022-11-16 08:38:02,48.700000
28,25.729297,16.086350,2022-11-16 08:40:17,2022-11-16 09:29:18,49.016667
29,23.947929,15.924788,2022-11-16 09:33:30,2022-11-16 10:21:07,47.616667
30,0.016667,0.006062,2022-11-16 10:24:16,2022-11-16 10:24:16,0.000000


In [391]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 60)& (rt["duration(min)"]>30)&(rt["energy(kwh)"] < 40)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='35'
final_df4['stop_id']='473'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/2930316655.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='35'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/2930316655.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='473'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,13.157383,7.982764,2022-11-01 17:34:46,2022-11-01 18:08:58,34.200000,35,473
3,22.103195,15.981981,2022-11-01 20:31:28,2022-11-01 21:12:23,40.916667,35,473
0,23.813069,16.052003,2022-11-03 19:52:44,2022-11-03 20:28:23,35.650000,35,473
1,23.897525,16.096686,2022-11-03 20:35:14,2022-11-03 21:08:21,33.116667,35,473
2,23.735186,16.052656,2022-11-03 21:18:42,2022-11-03 21:57:40,38.966667,35,473
...,...,...,...,...,...,...,...
24,22.918692,16.060645,2022-11-15 15:38:09,2022-11-15 16:22:34,44.416667,35,473
26,21.099588,16.057684,2022-11-16 06:58:29,2022-11-16 07:44:52,46.383333,35,473
27,24.183688,16.100508,2022-11-16 07:49:20,2022-11-16 08:38:02,48.700000,35,473
28,25.729297,16.086350,2022-11-16 08:40:17,2022-11-16 09:29:18,49.016667,35,473


In [392]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt35_new.csv",index=False,sep=',')

# rt 37

In [6]:
df2=pd.read_csv("rt37.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1403    33935
1207    21182
1012     3453
1200     3424
803      2172
1008     2012
1010     1687
705      1405
1013      759
1811      501
Name: vid, dtype: int64

In [119]:
df2=df3[df3['vid']==1403]
df2=df2[19000:20000]
#df2=pd.concat([df4,df5,df6,df7])

In [120]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="473"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [106]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,22.290802,17.358002,2022-11-15 10:10:02,2022-11-15 10:59:17,49.250000
0,21.381082,16.865489,2022-11-30 15:38:28,2022-11-30 16:39:39,61.183333
1,1.532163,0.962988,2022-12-01 08:23:53,2022-12-01 08:26:33,2.666667
0,23.250028,17.415609,2022-11-22 09:34:14,2022-11-22 10:12:14,38.000000
0,24.364689,17.081645,2022-12-08 09:40:57,2022-12-08 10:32:03,51.100000
1,13.822717,13.964872,2022-12-08 10:35:24,2022-12-08 14:08:53,213.483333
2,22.930296,17.061385,2022-12-08 14:11:23,2022-12-08 15:13:06,61.716667
3,23.088810,16.928019,2022-12-08 15:14:35,2022-12-08 16:07:46,53.183333
0,22.107943,17.288054,2022-11-17 17:29:26,2022-11-17 18:38:26,69.000000
0,23.011347,17.489815,2022-11-15 14:16:25,2022-11-15 15:12:25,56.000000


In [121]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 70)& (rt["duration(min)"]>30)&(rt["energy(kwh)"] < 30)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='37'
final_df4['stop_id']='473'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_8956/2429112962.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='37'
C:\Users\zheng\AppData\Local\Temp/ipykernel_8956/2429112962.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='473'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,22.290802,17.358002,2022-11-15 10:10:02,2022-11-15 10:59:17,49.250000,37,473
0,21.381082,16.865489,2022-11-30 15:38:28,2022-11-30 16:39:39,61.183333,37,473
0,23.250028,17.415609,2022-11-22 09:34:14,2022-11-22 10:12:14,38.000000,37,473
0,24.364689,17.081645,2022-12-08 09:40:57,2022-12-08 10:32:03,51.100000,37,473
2,22.930296,17.061385,2022-12-08 14:11:23,2022-12-08 15:13:06,61.716667,37,473
3,23.088810,16.928019,2022-12-08 15:14:35,2022-12-08 16:07:46,53.183333,37,473
0,22.107943,17.288054,2022-11-17 17:29:26,2022-11-17 18:38:26,69.000000,37,473
0,23.011347,17.489815,2022-11-15 14:16:25,2022-11-15 15:12:25,56.000000,37,473
1,24.507561,17.347636,2022-11-15 15:14:28,2022-11-15 16:13:14,58.766667,37,473
0,24.223250,17.476285,2022-11-03 17:16:27,2022-11-03 18:10:50,54.383333,37,473


In [122]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt37new.csv",index=False,sep=',')

# rt 38

In [123]:
df2=pd.read_csv("rt38.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2114    36595
1204    30847
1605    18586
1101    18188
1104    14548
1012     7386
1206     4524
1602     4449
1811     3293
1806     2881
1001     2510
1008     2185
1200     1777
2005     1728
1106     1286
569      1112
1607     1100
2111     1080
1016     1079
2580      912
1017      900
1007      892
804       885
2004      533
Name: vid, dtype: int64

In [215]:
df2=df3[df3['vid']==1104]

#df2=df2.head(1000)
df2=df2[8000:9000]
#df2=pd.concat([df4,df5])

In [216]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="925"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

KeyError: 'O'

In [210]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,15.550975,11.024190,2022-12-07 15:33:00,2022-12-07 16:25:44,52.733333
0,14.428670,10.996110,2022-11-29 08:41:05,2022-11-29 09:27:41,46.600000
1,16.769088,11.012843,2022-11-29 09:30:21,2022-11-29 10:18:20,47.983333
2,17.634713,14.803825,2022-11-29 10:23:16,2022-11-29 14:30:57,247.683333
0,15.673159,10.917652,2022-11-28 11:46:35,2022-11-28 12:35:17,48.700000
1,14.646359,10.897497,2022-11-28 12:37:05,2022-11-28 13:22:47,45.700000
2,16.495870,10.975514,2022-11-28 13:25:23,2022-11-28 14:14:19,48.933333
3,0.000000,0.000000,2022-12-07 06:59:12,2022-12-07 06:59:12,0.000000
4,17.621304,10.971947,2022-12-07 07:11:11,2022-12-07 07:42:23,31.200000
5,30.873683,22.057968,2022-12-07 07:46:05,2022-12-07 09:24:10,98.083333


In [217]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 60)& (rt["duration(min)"]>30)&(rt["energy(kwh)"] < 30)&(rt["energy(kwh)"] > 10)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='38'
final_df4['stop_id']='925'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_8956/2083845475.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='38'
C:\Users\zheng\AppData\Local\Temp/ipykernel_8956/2083845475.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='925'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,15.550975,11.024190,2022-12-07 15:33:00,2022-12-07 16:25:44,52.733333,38,925
0,14.428670,10.996110,2022-11-29 08:41:05,2022-11-29 09:27:41,46.600000,38,925
1,16.769088,11.012843,2022-11-29 09:30:21,2022-11-29 10:18:20,47.983333,38,925
0,15.673159,10.917652,2022-11-28 11:46:35,2022-11-28 12:35:17,48.700000,38,925
1,14.646359,10.897497,2022-11-28 12:37:05,2022-11-28 13:22:47,45.700000,38,925
2,16.495870,10.975514,2022-11-28 13:25:23,2022-11-28 14:14:19,48.933333,38,925
4,17.621304,10.971947,2022-12-07 07:11:11,2022-12-07 07:42:23,31.200000,38,925
0,13.999926,10.944536,2022-12-06 13:45:30,2022-12-06 14:28:46,43.266667,38,925
1,14.868740,10.788857,2022-12-06 14:37:23,2022-12-06 15:25:51,48.466667,38,925
3,15.920242,11.128370,2022-12-08 07:03:17,2022-12-08 07:42:18,39.016667,38,925


In [218]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt38new.csv",index=False,sep=',')

# rt43

In [219]:
df2=pd.read_csv("rt43.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1903    40073
2111    25117
2112     7954
1807     3214
1602     2160
1016     2082
1101     2042
1811     1856
2005     1633
Name: vid, dtype: int64

In [240]:
df2=df3[df3['vid']==2112]
df2=df2[2000:3000]
#df2=pd.concat([df4,df5,df6,df7,df8,df9,df10,df11])

In [241]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="660"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

ValueError: cannot join with no overlapping index names

In [373]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,49.333343,33.191010,2022-12-08 06:54:13,2022-12-08 08:13:51,79.633333
1,45.758028,32.995960,2022-12-08 08:24:19,2022-12-08 09:38:15,73.933333
2,51.119399,34.388600,2022-12-08 09:54:02,2022-12-08 14:16:06,262.066667
3,48.235266,33.086839,2022-12-08 14:24:33,2022-12-08 15:53:46,89.216667
0,50.408869,32.968107,2022-12-05 07:35:33,2022-12-05 08:51:52,76.316667
...,...,...,...,...,...
25,38.893075,41.437648,2022-11-29 09:55:54,2022-11-29 14:18:42,262.800000
26,47.391266,31.805167,2022-11-29 14:25:45,2022-11-29 15:46:51,81.100000
27,100.698422,66.719922,2022-11-29 15:55:05,2022-11-29 18:51:57,176.866667
28,50.296905,32.607167,2022-11-30 06:54:08,2022-11-30 08:16:49,82.683333


In [374]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 90)& (rt["duration(min)"]>60)&(rt["energy(kwh)"] < 60)&(rt["energy(kwh)"] > 30)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='43'
final_df4['stop_id']='520'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_26388/410722509.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='43'
C:\Users\zheng\AppData\Local\Temp/ipykernel_26388/410722509.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='520'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,49.333343,33.191010,2022-12-08 06:54:13,2022-12-08 08:13:51,79.633333,43,520
1,45.758028,32.995960,2022-12-08 08:24:19,2022-12-08 09:38:15,73.933333,43,520
3,48.235266,33.086839,2022-12-08 14:24:33,2022-12-08 15:53:46,89.216667,43,520
0,50.408869,32.968107,2022-12-05 07:35:33,2022-12-05 08:51:52,76.316667,43,520
1,52.348939,31.479438,2022-12-05 09:09:14,2022-12-05 10:33:04,83.833333,43,520
4,49.515461,32.473152,2022-12-07 06:10:13,2022-12-07 07:24:11,73.966667,43,520
5,50.917271,32.706684,2022-12-07 07:36:11,2022-12-07 08:51:48,75.616667,43,520
0,49.616204,32.678117,2022-11-15 07:37:45,2022-11-15 08:57:50,80.083333,43,520
1,47.237321,32.715214,2022-11-15 09:12:10,2022-11-15 10:31:14,79.066667,43,520
0,50.102130,33.299057,2022-11-14 06:56:25,2022-11-14 08:19:14,82.816667,43,520


In [375]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt43.csv",index=False,sep=',')

# rt 46

In [243]:
df1=pd.read_csv("rt46.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2001.0    25044
1806.0     1723
1807.0     1369
1016.0     1361
Name: vid, dtype: int64

In [300]:
df2=df3[df3['vid']==2001]
df2=df2[24000:25000]

In [301]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="473"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [277]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,7.644501,5.023966,2022-11-16 11:03:19,2022-11-16 14:18:45,195.433333
1,13.835186,8.393975,2022-11-16 14:22:59,2022-11-16 14:57:28,34.483333
2,12.287955,8.476490,2022-11-16 15:05:28,2022-11-16 15:42:48,37.333333
3,11.368692,8.280303,2022-11-16 15:50:29,2022-11-16 16:23:12,32.716667
0,10.020556,8.345573,2022-11-04 11:42:38,2022-11-04 12:21:36,38.966667
1,23.288879,16.714507,2022-11-04 12:29:26,2022-11-04 13:37:54,68.466667


# rt 75

In [393]:
df2=pd.read_csv("rt75.csv")
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

1205    40061
1401    31159
2574    12008
1016     8089
1017     3228
1007     2143
712      2013
803      1859
1806     1583
801      1332
2571     1254
702      1118
1008     1095
1206     1017
577       929
2578      923
1807      792
1200      696
1808      375
Name: vid, dtype: int64

In [501]:
df2=df3[df3['vid']==1200]
df2=df2[0:2000]

In [502]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="175"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

,energy(kwh),dist(km),start,end,duration(min)
0,56.42617,45.106253,2022-11-03 19:30:21,2022-11-03 21:18:35,108.233333


In [503]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,56.426170,45.106253,2022-11-03 19:30:21,2022-11-03 21:18:35,108.233333
0,0.025484,0.025408,2022-12-05 07:34:26,2022-12-05 07:34:26,0.000000
1,61.000136,45.005767,2022-12-05 07:42:40,2022-12-05 09:22:07,99.450000
0,22.591403,15.782186,2022-12-02 06:21:10,2022-12-02 07:12:12,51.033333
1,52.503867,45.125013,2022-12-02 07:40:19,2022-12-02 09:31:30,111.183333
0,54.020532,45.258694,2022-11-15 07:40:25,2022-11-15 09:32:42,112.283333
0,0.217828,0.078844,2022-12-07 10:22:22,2022-12-07 10:22:22,0.000000
1,0.035786,0.034289,2022-12-07 14:03:53,2022-12-07 14:03:53,0.000000
2,61.128462,45.017016,2022-12-07 14:23:46,2022-12-07 16:16:18,112.533333
3,64.376313,45.076691,2022-12-07 16:30:04,2022-12-07 18:30:44,120.666667


In [504]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 110)& (rt["duration(min)"]>40)&(rt["energy(kwh)"] < 60)&(rt["energy(kwh)"] > 30)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='75'
final_df4['stop_id']='175'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/2946618783.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='75'
C:\Users\zheng\AppData\Local\Temp/ipykernel_18668/2946618783.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='175'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,56.426170,45.106253,2022-11-03 19:30:21,2022-11-03 21:18:35,108.233333,75,175
0,54.891704,44.272006,2022-11-17 06:24:44,2022-11-17 08:14:34,109.833333,75,175
1,53.311715,44.296092,2022-11-17 08:23:21,2022-11-17 10:06:42,103.350000,75,175
0,59.493671,45.173308,2022-11-03 07:00:18,2022-11-03 08:42:42,102.400000,75,175
2,52.787693,41.718708,2022-11-01 17:28:04,2022-11-01 19:08:55,100.850000,75,175
4,57.890346,44.895285,2022-11-02 07:00:00,2022-11-02 08:45:56,105.933333,75,175
6,57.337550,45.041709,2022-11-04 11:15:06,2022-11-04 13:03:37,108.516667,75,175
12,55.297868,44.610557,2022-11-04 09:25:09,2022-11-04 10:54:08,88.983333,75,175
17,58.298583,45.154262,2022-11-07 07:00:57,2022-11-07 08:45:23,104.433333,75,175


In [505]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt75_new.csv",index=False,sep=',')

# 处理异常值

# rt46

In [383]:
df1=pd.read_csv("rt46.csv")
df1=df1.iloc[:,:24]
df3=df1.groupby(by='vid').apply(cal)
vidlist = df3['vid'].unique().tolist()
vidlist

[1016.0, 1806.0, 1807.0, 2001.0]

In [457]:
df2=df3[df3['vid']==2001]

df4=df2[2000:3000]
df5=df2[5000:8000]
df6=df2[9000:11000]
df7=df2[12000:14000]
df8=df2[15000:17000]
df9=df2[20000:22000]
df10=df2[23000:24000]

df2=pd.concat([df4,df5,df6,df7,df8,df9,df10])

In [458]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

,energy(kwh),dist(km),start,end,duration(min)
0,19.532360,14.880832,2022-11-03 12:10:29,2022-11-03 14:07:32,117.050000
1,11.474821,7.653077,2022-11-03 14:14:23,2022-11-03 14:45:15,30.866667
2,11.949573,13.560165,2022-11-03 15:03:26,2022-11-03 16:59:03,115.616667
3,13.496069,8.398082,2022-11-04 08:45:23,2022-11-04 09:15:44,30.350000
4,11.583751,8.345103,2022-11-07 10:47:16,2022-11-07 11:24:51,37.583333
5,23.723229,16.599497,2022-11-07 11:30:24,2022-11-07 12:45:46,75.366667
6,36.435181,25.251858,2022-11-07 12:57:31,2022-11-07 14:37:23,99.866667
7,10.913099,8.292325,2022-11-08 08:05:16,2022-11-08 08:35:42,30.433333
8,13.548797,8.288613,2022-11-08 08:49:20,2022-11-08 09:19:28,30.133333
9,11.942421,8.268951,2022-11-08 09:27:50,2022-11-08 10:02:35,34.750000


In [459]:
final_df4=pd.concat([final_df3,final_df4])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,19.532360,14.880832,2022-11-03 12:10:29,2022-11-03 14:07:32,117.050000
1,11.474821,7.653077,2022-11-03 14:14:23,2022-11-03 14:45:15,30.866667
2,11.949573,13.560165,2022-11-03 15:03:26,2022-11-03 16:59:03,115.616667
3,13.496069,8.398082,2022-11-04 08:45:23,2022-11-04 09:15:44,30.350000
4,11.583751,8.345103,2022-11-07 10:47:16,2022-11-07 11:24:51,37.583333
5,23.723229,16.599497,2022-11-07 11:30:24,2022-11-07 12:45:46,75.366667
6,36.435181,25.251858,2022-11-07 12:57:31,2022-11-07 14:37:23,99.866667
7,10.913099,8.292325,2022-11-08 08:05:16,2022-11-08 08:35:42,30.433333
8,13.548797,8.288613,2022-11-08 08:49:20,2022-11-08 09:19:28,30.133333
9,11.942421,8.268951,2022-11-08 09:27:50,2022-11-08 10:02:35,34.750000


In [462]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 55)& (rt["duration(min)"]>25)&(rt["energy(kwh)"] > 7)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='46'
final_df4['stop_id']='1'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_19528/2618243956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='46'
C:\Users\zheng\AppData\Local\Temp/ipykernel_19528/2618243956.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='1'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
1,11.474821,7.653077,2022-11-03 14:14:23,2022-11-03 14:45:15,30.866667,46,1
3,13.496069,8.398082,2022-11-04 08:45:23,2022-11-04 09:15:44,30.350000,46,1
4,11.583751,8.345103,2022-11-07 10:47:16,2022-11-07 11:24:51,37.583333,46,1
7,10.913099,8.292325,2022-11-08 08:05:16,2022-11-08 08:35:42,30.433333,46,1
8,13.548797,8.288613,2022-11-08 08:49:20,2022-11-08 09:19:28,30.133333,46,1
9,11.942421,8.268951,2022-11-08 09:27:50,2022-11-08 10:02:35,34.750000,46,1
10,12.291752,8.318611,2022-11-08 10:08:35,2022-11-08 10:41:25,32.833333,46,1
12,12.404067,8.376559,2022-11-08 14:20:06,2022-11-08 14:51:03,30.950000,46,1
13,11.434344,8.271184,2022-11-08 14:59:22,2022-11-08 15:35:54,36.533333,46,1
14,11.682673,8.294769,2022-11-09 08:49:52,2022-11-09 09:18:49,28.950000,46,1


In [465]:
final_df4.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt46.csv",index=False,sep=',')

In [4]:
def OD_identify(intersect):
    spdList=list(intersect["spd"])
    dest=[]
    
    for i in range(len(spdList)):
        if spdList[i-1]!=0 and spdList[i]==0 and spdList[i+1:i+5]==[0.0, 0.0, 0.0, 0.0]:
            dest.append('D')
        elif spdList[i]!=0 and spdList[i-4:i]==[0.0, 0.0, 0.0, 0.0]:
            dest.append('O')
        else: 
            dest.append('0')
    
    #append one OD identify column
    dest = pd.DataFrame(dest, columns =['O/D']).reset_index(drop=True)
    intersect=pd.concat([intersect,dest],axis=1)
    
    return intersect

# first step to split by journey 
def OD_identify2(intersect1,df):
    OD1=OD_identify(intersect1)
    OD=OD1[['id','O/D']]#replace OD2=pd.concat([OD2_1,OD2_2]) because of round trip
    #append the OD info to the origin datasets
    rt_OD = pd.merge(df, OD, how='left', on=['id'])
    rt_OD.set_index(["id"], inplace=True)
    rt_O=rt_OD[rt_OD['O/D']=="O"]
#identify the first origin
    rt_start=rt_O.iloc[[0],[42]].values[0][0]
#identify the last dest
    rt_D=rt_OD[rt_OD['O/D']=="D"]
    rt_end=rt_D.iloc[[-1],[42]].values[0][0]
    
    return rt_OD.loc[rt_start:rt_end]

# 定义通用的获取某元素在列表中第n次出现的位置下标的函数
def get_index(l, x, n):
    if n <= l.count(x):
        all_index = [key for key, value in enumerate(l) if value == x]
        return all_index[n]#change[n-1] to n
    else:
        return None

# rt 127

In [6]:
df1=pd.read_csv("rt127.csv")
df1=df1.iloc[:,:24]
df3=df1.groupby(by='vid').apply(cal)
vidlist = df3['vid'].unique().tolist()
df1['vid'].value_counts()

1015.0    6233
2572.0    3475
702.0      163
Name: vid, dtype: int64

In [48]:
df2=df3[df3['vid']==2572]
df2=df2[2000:3000]
#df2=pd.concat([df4,df5,df6])

In [49]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer_merge.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1192"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

IndexError: positional indexers are out-of-bounds

In [67]:
final_df4=pd.concat([final_df3,final_df2])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,4.438482,3.449821,2022-11-07 12:27:32,2022-11-07 12:52:52,25.333333
1,4.213314,3.530174,2022-11-07 12:58:43,2022-11-07 13:22:28,23.750000
2,5.575875,3.745305,2022-11-07 13:27:49,2022-11-07 13:53:13,25.400000
3,7.056428,6.674679,2022-11-07 14:00:41,2022-11-07 14:51:01,50.333333
4,0.364442,0.231378,2022-11-07 15:22:30,2022-11-07 15:22:36,0.100000
...,...,...,...,...,...
54,4.438427,4.025295,2022-11-22 16:01:15,2022-11-22 16:21:16,20.016667
55,4.727190,4.002625,2022-11-22 16:31:01,2022-11-22 16:50:20,19.316667
56,5.759749,4.019501,2022-11-22 17:00:29,2022-11-22 17:21:26,20.950000
57,4.105547,4.094380,2022-11-22 17:26:34,2022-11-22 17:51:06,24.533333


In [88]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 35)& (rt["duration(min)"]>15)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='127'
final_df4['stop_id']='1238'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_41300/2440833983.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='127'
C:\Users\zheng\AppData\Local\Temp/ipykernel_41300/2440833983.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='1238'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,4.438482,3.449821,2022-11-07 12:27:32,2022-11-07 12:52:52,25.333333,127,1238
1,4.213314,3.530174,2022-11-07 12:58:43,2022-11-07 13:22:28,23.750000,127,1238
2,5.575875,3.745305,2022-11-07 13:27:49,2022-11-07 13:53:13,25.400000,127,1238
5,4.793592,3.659992,2022-11-07 15:29:47,2022-11-07 15:53:10,23.383333,127,1238
1,4.461293,3.479859,2022-11-03 13:56:13,2022-11-03 14:18:54,22.683333,127,1238
3,5.513611,4.023970,2022-11-03 15:29:38,2022-11-03 15:52:32,22.900000,127,1238
4,5.147383,3.993584,2022-11-03 15:56:53,2022-11-03 16:21:08,24.250000,127,1238
5,4.754537,4.046028,2022-11-03 16:26:09,2022-11-03 16:51:43,25.566667,127,1238
6,4.838803,4.013054,2022-11-03 16:57:05,2022-11-03 17:22:59,25.900000,127,1238
7,5.336495,4.045442,2022-11-03 17:29:26,2022-11-03 17:51:34,22.133333,127,1238


In [89]:
final_df2.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt127.csv",index=False,sep=',')

# rt 150

In [14]:
df1=pd.read_csv("rt150.csv")
df1=df1.iloc[:,:24]
df3=df1.groupby(by='vid').apply(cal)
vidlist = df3['vid'].unique().tolist()
df1['vid'].value_counts()

4380.0    3082
4398.0    1548
4379.0    1259
4378.0     814
Name: vid, dtype: int64

In [45]:
df2=df3[df3['vid']==4378]
#
#df4=df2[1000:4000]
#df2=df2[0:1000]
#df2=pd.concat([df4,df5])

In [46]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1563"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df3=DataFrame(final_output2)#将字典转换成为数据框
final_df3

C:\Users\zheng\AppData\Local\Temp/ipykernel_31016/1812503939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id'] = range(len(df2))
C:\Users\zheng\AppData\Local\Temp/ipykernel_31016/1812503939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id1'] = range(len(df2))


,energy(kwh),dist(km),start,end,duration(min)
0,46.304087,23.588944,2022-10-28 11:01:57,2022-10-28 11:55:30,53.55


In [71]:
final_df2

,energy(kwh),dist(km),start,end,duration(min)
0,32.446975,23.741604,2022-11-04 14:00:21,2022-11-04 14:57:37,57.266667
1,0.000000,0.000000,2022-11-04 15:00:52,2022-11-04 15:04:30,3.633333
2,40.845956,23.293366,2022-11-07 08:51:54,2022-11-07 13:00:05,248.183333
3,31.637188,23.585802,2022-11-07 13:02:08,2022-11-07 13:51:28,49.333333


In [108]:
final_df4=pd.concat([final_df3,final_df2])
final_df4=final_df4.drop_duplicates()
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,35.160147,23.575359,2022-11-03 10:01:42,2022-11-03 10:48:10,46.466667
1,35.668760,23.795016,2022-11-03 11:01:19,2022-11-03 11:49:13,47.900000
2,43.999597,23.740021,2022-11-07 09:01:55,2022-11-07 09:50:11,48.266667
3,40.557613,23.670794,2022-11-07 10:00:36,2022-11-07 10:44:51,44.250000
4,35.131329,23.536628,2022-11-07 11:01:58,2022-11-07 11:48:31,46.550000
5,28.575024,22.872281,2022-11-08 08:03:30,2022-11-08 08:51:14,47.733333
6,35.907691,23.479098,2022-11-08 09:06:13,2022-11-08 09:51:21,45.133333
7,35.617118,23.721699,2022-11-08 10:01:24,2022-11-08 10:50:11,48.783333
8,33.827829,23.385965,2022-11-09 09:00:51,2022-11-09 09:49:57,49.100000
9,35.107271,23.512484,2022-11-09 10:00:38,2022-11-09 10:47:24,46.766667


In [109]:
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 65)& (rt["duration(min)"]>45)]
    return rt_filtered

final_df4=clean_df(final_df4)
final_df4['rt']='127'
final_df4['stop_id']='1563'
final_df4

C:\Users\zheng\AppData\Local\Temp/ipykernel_3644/2722330785.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['rt']='127'
C:\Users\zheng\AppData\Local\Temp/ipykernel_3644/2722330785.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['stop_id']='1563'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,35.160147,23.575359,2022-11-03 10:01:42,2022-11-03 10:48:10,46.466667,127,1563
1,35.668760,23.795016,2022-11-03 11:01:19,2022-11-03 11:49:13,47.900000,127,1563
2,43.999597,23.740021,2022-11-07 09:01:55,2022-11-07 09:50:11,48.266667,127,1563
4,35.131329,23.536628,2022-11-07 11:01:58,2022-11-07 11:48:31,46.550000,127,1563
5,28.575024,22.872281,2022-11-08 08:03:30,2022-11-08 08:51:14,47.733333,127,1563
6,35.907691,23.479098,2022-11-08 09:06:13,2022-11-08 09:51:21,45.133333,127,1563
7,35.617118,23.721699,2022-11-08 10:01:24,2022-11-08 10:50:11,48.783333,127,1563
8,33.827829,23.385965,2022-11-09 09:00:51,2022-11-09 09:49:57,49.100000,127,1563
9,35.107271,23.512484,2022-11-09 10:00:38,2022-11-09 10:47:24,46.766667,127,1563
10,30.659297,22.276441,2022-11-15 08:02:15,2022-11-15 08:48:23,46.133333,127,1563


# rt 126

In [49]:
df1=pd.read_csv("rt126.csv")
df1=df1.iloc[:,:24]
df3=df1.groupby(by='vid').apply(cal)
vidlist = df3['vid'].unique().tolist()
vidlist

[575.0, 706.0, 709.0, 1006.0, 1103.0, 1200.0, 1206.0, 2102.0, 2574.0]

In [25]:
df1['vid'].value_counts()

2574.0    1200
2102.0     869
1006.0     214
1200.0     170
1103.0     167
575.0      160
706.0      152
1206.0     149
709.0       98
Name: vid, dtype: int64

In [73]:
df2=df3[df3['vid']==2102]

In [74]:


#根据时间排序
#df2=df2.sort_values(by = 'tmstmp')
#df2=pd.read_csv("rt2.csv")
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1238"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    OD1=OD_identify(intersects['inter' + str(i)])

    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df2=DataFrame(final_output2)#将字典转换成为数据框
final_df2

C:\Users\zheng\AppData\Local\Temp/ipykernel_31016/3314417719.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id'] = range(len(df2))
C:\Users\zheng\AppData\Local\Temp/ipykernel_31016/3314417719.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['id1'] = range(len(df2))


,energy(kwh),dist(km),start,end,duration(min)
0,15.374387,10.126760,2022-10-23 14:55:07,2022-10-23 15:31:11,36.066667
1,15.800609,10.273983,2022-10-23 15:33:59,2022-10-23 16:13:27,39.466667
2,15.338810,10.260871,2022-10-23 16:15:28,2022-10-23 16:50:59,35.516667
3,16.007472,10.257488,2022-10-23 16:55:54,2022-10-23 17:33:39,37.750000


# rt 122

In [7]:
df1=pd.read_csv("rt122.csv")
df1=df1.iloc[:,:24]
df3=df1.groupby(by='vid').apply(cal)
vidlist = df3['vid'].unique().tolist()
vidlist

[573.0, 705.0, 712.0, 1811.0, 2003.0, 2005.0, 2111.0, 2601.0]

In [9]:
df3.to_csv("rt122groupby.csv")

In [7]:
df1['vid'].value_counts()

2601.0    2293
2003.0     747
1811.0     586
2111.0     404
712.0      336
2005.0     269
705.0      137
573.0      131
Name: vid, dtype: int64

In [12]:
df2=df3[df3['vid']==2601]
df2=df2[1000:2293]
#df2=pd.concat([df4,df5])

In [11]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1196"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)

    #change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df2=DataFrame(final_output2)#将字典转换成为数据框
final_df2

IndexError: positional indexers are out-of-bounds

# rt 28

In [134]:
df1=pd.read_csv("rt78.csv")
df2=df1.iloc[:,:24]
df3=df2.groupby(by='vid').apply(cal)
vidlist = df2['vid'].unique().tolist()
df2['vid'].value_counts()

2004.0    4106
Name: vid, dtype: int64

In [135]:
df2=df1[df1['vid']==2004]
#df2=df2[1000:4000]

In [138]:
df2['id'] = range(len(df2))
df2['id1'] = range(len(df2))

datelist = df2['stsd'].unique() #divide the df into sub df based on date
#!!delete this code later, just for testing the calculation by date


gdf2 = geopandas.GeoDataFrame(df2, geometry=geopandas.points_from_xy(df2.lon, df2.lat))
gdf2.crs = pyproj.CRS.from_user_input('EPSG:4326') 

shp_path="C:/Users/zheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stopBuffer.shp"
shp_df = geopandas.GeoDataFrame.from_file(shp_path, )
shp_df.crs = pyproj.CRS.from_user_input('EPSG:4326') 

#revise this when switch to others
stop2_1=shp_df[shp_df['stop_id']=="1493"]

names = locals()
gdfs = locals()
intersects=locals()
trips=locals()
OD_count=locals()
O_count=locals()
D_count=locals()
ODToList = locals()
energy_list=[]
dist_list=[]
start_list=[]
end_list=[]
duration_list=[]

for i in range(len(datelist)):    
    names['n' + str(i)]=df2[df2['stsd'].isin([datelist[i]])]#store into multiple df
    gdfs['gdf' + str(i)]=geopandas.GeoDataFrame(names['n' + str(i)], geometry=geopandas.points_from_xy(names['n' + str(i)].lon, names['n' + str(i)].lat))
    gdfs['gdf' + str(i)].crs = pyproj.CRS.from_user_input('EPSG:4326') 
    intersects['inter' + str(i)]= geopandas.overlay(gdfs['gdf' + str(i)], stop2_1, how='intersection')
    trips["trips"+ str(i)]=OD_identify2(intersects['inter' + str(i)],names['n' + str(i)])
    OD_count['OD_count'+ str(i)]=trips["trips"+ str(i)]['O/D'].value_counts()
    O_count['O_count'+ str(i)]=OD_count['OD_count'+ str(i)]['O']
    D_count['D_count'+ str(i)]=OD_count['OD_count'+ str(i)]['D']
    ODToList['ODToList'+ str(i)] = list(trips["trips"+ str(i)]['O/D'])
    
    energy_list1=[]
    dist_list1=[]
    start_list1=[]
    end_list1=[]
    duration_list1=[]
    for j in range(0,O_count['O_count'+ str(i)]):
        O_index = get_index(ODToList['ODToList'+ str(i)], "O", j)#result: 0
        D_index = get_index(ODToList['ODToList'+ str(i)], "D", j)
        rt_energy=trips["trips"+ str(i)][O_index:D_index]['Etr'].sum()
        rt_dist=trips["trips"+ str(i)][O_index:D_index]['dist'].sum()
        #get the start/end time
        #print(D_index)
        #Q:can't print the time since 20221122 16:55:30
        rt_start=trips["trips"+ str(i)][O_index:D_index].iloc[[0]].values[0][2]#第1行第3列的值
        rt_end=trips["trips"+ str(i)][O_index:D_index].iloc[[-1]].values[0][2]#最后1行第3列的值
        rt_start=pd.to_datetime(rt_start)
        rt_end=pd.to_datetime(rt_end)
        rt_duration=rt_end-rt_start
        #append to list
        duration_list1.append(rt_duration.total_seconds()/60) #save in min
        energy_list1.append(rt_energy)
        dist_list1.append(rt_dist)
        start_list1.append(rt_start)
        end_list1.append(rt_end)
    duration_list.extend(duration_list1)
    energy_list.extend(energy_list1)
    dist_list.extend(dist_list1)
    start_list.extend(start_list1)
    end_list.extend(end_list1)


#change the output name
final_output2={"energy(kwh)" : energy_list,
   "dist(km)" : dist_list,
         "start":start_list,
         "end":end_list,
        "duration(min)":duration_list}#将列表转换成字典
final_df2=DataFrame(final_output2)#将字典转换成为数据框
final_df2

,energy(kwh),dist(km),start,end,duration(min)
0,42.907177,27.739408,2022-12-12 08:56:07,2022-12-12 10:00:00,63.883333
1,40.185878,27.774639,2022-12-13 07:16:44,2022-12-13 08:30:16,73.533333
2,41.886389,27.716353,2022-12-13 08:47:41,2022-12-13 10:00:46,73.083333
3,42.143110,27.918264,2022-12-14 07:17:10,2022-12-14 08:32:40,75.500000
4,41.134255,27.434345,2022-12-14 08:51:08,2022-12-14 09:56:07,64.983333
5,39.657036,27.981554,2022-12-15 07:17:14,2022-12-15 08:35:11,77.950000
6,42.308931,27.683063,2022-12-15 08:55:16,2022-12-15 09:58:00,62.733333
7,39.561775,27.828466,2022-12-16 08:48:37,2022-12-16 09:52:13,63.600000


In [131]:
final_df4=pd.concat([final_df3,final_df2])
final_df4

,energy(kwh),dist(km),start,end,duration(min)
0,37.825424,25.919172,2022-12-16 08:00:01,2022-12-16 08:50:08,50.116667
1,34.484511,25.508795,2022-12-16 09:02:26,2022-12-16 09:48:46,46.333333
2,0.012890,0.016506,2022-12-16 09:54:31,2022-12-16 09:54:31,0.000000
3,33.580995,25.180599,2022-12-16 10:00:24,2022-12-16 10:48:05,47.683333
0,0.170913,0.187330,2022-12-13 09:50:56,2022-12-13 09:51:10,0.233333
1,39.981079,26.225942,2022-12-13 09:58:22,2022-12-13 10:48:06,49.733333
2,33.451532,25.968035,2022-12-14 08:00:29,2022-12-14 08:53:41,53.200000
3,39.401901,25.995439,2022-12-14 09:02:43,2022-12-14 09:51:43,49.000000
4,36.893725,25.689432,2022-12-14 10:00:23,2022-12-14 10:54:00,53.616667
5,36.536711,26.047246,2022-12-15 07:59:45,2022-12-15 08:50:56,51.183333


In [132]:
#this function varies depending on the intervals of bus!!
def clean_df(rt):
    rt_filtered = rt[(rt["duration(min)"] < 60)& (rt["duration(min)"]>40)]
    return rt_filtered

final_df2=clean_df(final_df2)
final_df2['rt']='76'
final_df2['stop_id']='520'
final_df2

C:\Users\zheng\AppData\Local\Temp/ipykernel_24896/3326111989.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['rt']='76'
C:\Users\zheng\AppData\Local\Temp/ipykernel_24896/3326111989.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['stop_id']='520'


,energy(kwh),dist(km),start,end,duration(min),rt,stop_id
0,37.825424,25.919172,2022-12-16 08:00:01,2022-12-16 08:50:08,50.116667,76,520
1,34.484511,25.508795,2022-12-16 09:02:26,2022-12-16 09:48:46,46.333333,76,520
3,33.580995,25.180599,2022-12-16 10:00:24,2022-12-16 10:48:05,47.683333,76,520
1,39.981079,26.225942,2022-12-13 09:58:22,2022-12-13 10:48:06,49.733333,76,520
2,33.451532,25.968035,2022-12-14 08:00:29,2022-12-14 08:53:41,53.200000,76,520
3,39.401901,25.995439,2022-12-14 09:02:43,2022-12-14 09:51:43,49.000000,76,520
4,36.893725,25.689432,2022-12-14 10:00:23,2022-12-14 10:54:00,53.616667,76,520
5,36.536711,26.047246,2022-12-15 07:59:45,2022-12-15 08:50:56,51.183333,76,520
6,32.948804,24.884907,2022-12-15 09:00:05,2022-12-15 09:52:17,52.200000,76,520
7,32.285884,25.246777,2022-12-15 10:00:50,2022-12-15 10:52:14,51.400000,76,520


In [139]:
final_df2.to_csv("C:/Users/zheng/Dropbox (UFL)/Gap analysis/result/rt78new.csv",index=False,sep=',')